# ***Imports***

In [1]:
!pip install -q textstat
!pip install -q faiss-cpu
!pip install -q chromadb
!pip install -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.1/239.1 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.7/939.7 kB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 118.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.8 MB/s et

In [2]:
#Importing required libraries
import os
import json
import sqlite3
import logging
import datetime
import pickle
import re
import threading
import time
from typing import List, Dict, Any, Optional, Tuple, Union
from dataclasses import dataclass, asdict
from pathlib import Path
from collections import defaultdict, deque
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM,pipeline, BitsAndBytesConfig, GenerationConfig
from sentence_transformers import SentenceTransformer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob
import textstat
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import faiss
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from wordcloud import WordCloud
import gradio as gr
import chromadb
from chromadb.config import Settings
from datetime import datetime, timedelta
import uuid
import hashlib
import asyncio
from concurrent.futures import ThreadPoolExecutor
import queue
from huggingface_hub import login

import warnings
warnings.filterwarnings('ignore')

In [3]:
#Downloading required NLTK data
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('vader_lexicon', quiet=True)
nltk.download('wordnet', quiet=True)

True

In [4]:
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

GPU: NVIDIA A100-SXM4-40GB
GPU Memory: 42.5 GB


In [5]:
login()

# ***Configuration and Data Models***

In [6]:
@dataclass
class ChatbotConfig:
    """Advanced configuration for the sophisticated chatbot system"""

    # Model configurations - Smart resource management for A100 40GB
    PRIMARY_MODEL = "mistralai/Mistral-7B-Instruct-v0.3"  # Main model (~14GB with 4-bit)
    EMBEDDING_MODEL = "sentence-transformers/all-mpnet-base-v2"  # Lightweight (~420MB)
    SENTIMENT_MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"  # Lightweight (~500MB)
    EMOTION_MODEL = "j-hartmann/emotion-english-distilroberta-base"  # Emotion detection (~500MB)
    LLM_MODEL = "microsoft/DialoGPT-large"

    # Alternative models for specialized tasks (loaded on demand)
    FALLBACK_MODEL = "microsoft/DialoGPT-large"  # If memory constraints
    CODE_MODEL_NAME = "codellama/CodeLlama-7b-Instruct-hf"  # For code tasks only
    LARGE_MODEL_NAME = "meta-llama/Llama-2-13b-chat-hf"  # For complex tasks only

    # Memory management strategy
    USE_MODEL_SWITCHING = True   # Load models on demand
    UNLOAD_INACTIVE_MODELS = True  # Free memory when switching
    MAX_CONCURRENT_MODELS = 2    # Primary + embedding model typically

    # Model loading configurations
    USE_4BIT_QUANTIZATION = True  # Enables more models in memory
    USE_8BIT_QUANTIZATION = False
    LOAD_IN_HALF_PRECISION = True
    DEVICE_MAP = "auto"

    # Memory system parameters
    MAX_MEMORY_SIZE = 10000  # Maximum conversations to store
    CONTEXT_WINDOW = 4096    # Token limit for context
    SIMILARITY_THRESHOLD = 0.7  # For memory retrieval
    MEMORY_DECAY_FACTOR = 0.95  # How much old memories fade

    # RAG system parameters
    CHUNK_SIZE = 512         # Document chunk size
    CHUNK_OVERLAP = 50       # Overlap between chunks
    TOP_K_RETRIEVAL = 5      # Number of relevant chunks to retrieve
    EMBEDDING_DIMENSION = 768  # Updated for all-mpnet-base-v2

    # Personality system
    PERSONALITY_BLEND_WEIGHT = 0.3  # How much personality affects responses
    ADAPTATION_RATE = 0.1    # How quickly personality adapts

    # Technical parameters - Optimized for advanced models
    MAX_NEW_TOKENS = 512     # Increased for better responses
    TEMPERATURE = 0.7
    TOP_P = 0.9
    TOP_K = 50
    REPETITION_PENALTY = 1.15
    DO_SAMPLE = True
    PAD_TOKEN_ID = None      # Will be set dynamically

    # Storage paths (Google Drive integration)
    MEMORY_DB_PATH = "/content/drive/MyDrive/chatbot_memory.db"
    KNOWLEDGE_BASE_PATH = "/content/drive/MyDrive/knowledge_base/"
    PERSONALITIES_PATH = "/content/drive/MyDrive/personalities.json"
    CONVERSATION_LOGS = "/content/drive/MyDrive/conversation_logs/"
    EMBEDDINGS_PATH = "/content/drive/MyDrive/embeddings/"

In [7]:
@dataclass
class Message:
    """Represents a single message in conversation"""
    id: str
    user_id: str
    content: str
    timestamp: datetime
    message_type: str  # 'user' or 'assistant'
    sentiment_score: float
    emotion: str
    personality_used: str
    context_sources: List[str]
    metadata: Dict[str, Any]

    def to_dict(self) -> Dict[str, Any]:
        """Convert message to dictionary for storage"""
        data = asdict(self)
        data['timestamp'] = self.timestamp.isoformat()
        return data

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Message':
        """Create message from dictionary"""
        data['timestamp'] = datetime.fromisoformat(data['timestamp'])
        return cls(**data)

In [8]:
@dataclass
class UserProfile:
    """Comprehensive user profile with preferences and history"""
    user_id: str
    name: str
    preferred_personality: str
    conversation_style: str  # formal, casual, technical, friendly
    topics_of_interest: List[str]
    sentiment_history: List[float]
    personality_preferences: Dict[str, float]
    learning_goals: List[str]
    interaction_patterns: Dict[str, Any]
    created_at: datetime
    last_active: datetime

    def to_dict(self) -> Dict[str, Any]:
        """Convert profile to dictionary for storage"""
        data = asdict(self)
        data['created_at'] = self.created_at.isoformat()
        data['last_active'] = self.last_active.isoformat()
        return data

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'UserProfile':
        """Create profile from dictionary"""
        data['created_at'] = datetime.fromisoformat(data['created_at'])
        data['last_active'] = datetime.fromisoformat(data['last_active'])
        return cls(**data)

In [9]:
@dataclass
class ConversationMemory:
    """Represents a complete conversation with context and metadata"""
    conversation_id: str
    user_id: str
    messages: List[Message]
    summary: str
    key_topics: List[str]
    overall_sentiment: float
    personality_evolution: Dict[str, float]
    importance_score: float
    created_at: datetime
    updated_at: datetime

    def add_message(self, message: Message):
        """Add a new message to the conversation"""
        self.messages.append(message)
        self.updated_at = datetime.now()

    def get_conversation_text(self) -> str:
        """Get full conversation as formatted text"""
        text_parts = []
        for msg in self.messages:
            prefix = "User" if msg.message_type == "user" else "Assistant"
            text_parts.append(f"{prefix}: {msg.content}")
        return "\n".join(text_parts)

In [10]:
@dataclass
class PersonalityProfile:
    """Defines a specific personality for the chatbot - Simple and explainable"""
    name: str
    description: str
    traits: Dict[str, float]  # personality dimensions (0-1) - Easy to visualize
    system_prompt: str        # Clear prompt engineering approach
    temperature: float        # Technical parameter that affects creativity
    top_p: float             # Technical parameter that affects response diversity

    def get_prompt_prefix(self) -> str:
        """Generate personality-specific prompt prefix"""
        return f"{self.system_prompt}\n\nRespond with {self.traits['formality']*100:.0f}% formality and {self.traits['friendliness']*100:.0f}% friendliness."

In [11]:
class AdvancedLogger:
    """Professional logging system for the chatbot"""

    def __init__(self, log_level=logging.INFO):
        self.logger = logging.getLogger('SophisticatedChatbot')

        # Prevent duplicate handlers
        if self.logger.handlers:
            self.logger.handlers.clear()

        self.logger.setLevel(log_level)
        self.logger.propagate = False  # Prevent duplicate logging

        # Create formatter
        formatter = logging.Formatter(
            '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        )

        # Console handler
        console_handler = logging.StreamHandler()
        console_handler.setFormatter(formatter)
        self.logger.addHandler(console_handler)

        # File handler (if Google Drive is mounted)
        try:
            file_handler = logging.FileHandler('/content/drive/MyDrive/chatbot.log')
            file_handler.setFormatter(formatter)
            self.logger.addHandler(file_handler)
        except:
            pass  # Google Drive not mounted yet

    def info(self, message: str):
        self.logger.info(message)

    def error(self, message: str):
        self.logger.error(message)

    def warning(self, message: str):
        self.logger.warning(message)

    def debug(self, message: str):
        self.logger.debug(message)

In [12]:
# Initialize global logger
logger = AdvancedLogger()

In [13]:
# Simplified personality profiles - Easy to explain and demonstrate
DEFAULT_PERSONALITIES = {
    "professional": PersonalityProfile(
        name="Professional Assistant",
        description="Formal, analytical, and structured responses",
        traits={
            "formality": 0.9,
            "friendliness": 0.6,
            "creativity": 0.4,
            "analytical": 0.9
        },
        system_prompt="You are a professional AI assistant. Provide structured, analytical responses with clear reasoning.",
        temperature=0.3,  # Lower creativity for consistency
        top_p=0.8
    ),

    "friendly": PersonalityProfile(
        name="Friendly Companion",
        description="Warm, conversational, and empathetic responses",
        traits={
            "formality": 0.3,
            "friendliness": 0.9,
            "creativity": 0.7,
            "analytical": 0.6
        },
        system_prompt="You are a friendly and supportive AI companion. Be warm, conversational, and encouraging.",
        temperature=0.7,  # Higher creativity for natural conversation
        top_p=0.9
    ),

    "creative": PersonalityProfile(
        name="Creative Innovator",
        description="Imaginative, innovative, and inspiring responses",
        traits={
            "formality": 0.4,
            "friendliness": 0.8,
            "creativity": 0.95,
            "analytical": 0.5
        },
        system_prompt="You are a creative AI that thinks outside the box. Provide innovative ideas and inspiring perspectives.",
        temperature=0.9,  # High creativity for innovation
        top_p=0.95
    ),

    "analytical": PersonalityProfile(
        name="Data Scientist",
        description="Logical, precise, and evidence-based responses",
        traits={
            "formality": 0.8,
            "friendliness": 0.5,
            "creativity": 0.3,
            "analytical": 0.95
        },
        system_prompt="You are an analytical AI focused on data-driven insights. Provide logical, evidence-based responses.",
        temperature=0.2,  # Very low creativity for precision
        top_p=0.7
    )
}

# ***Advanced Memory System***

In [14]:
class DatabaseManager:
    """Handles all database operations for persistent memory"""

    def __init__(self, db_path: str):
        self.db_path = self._ensure_database_path(db_path)
        self.init_database()

    def _ensure_database_path(self, db_path: str) -> str:
        """Ensure database path is accessible, with fallback options"""
        try:
            # Try to create directory if it doesn't exist
            db_dir = os.path.dirname(db_path)

            # If Google Drive path, try to mount it
            if "/content/drive/" in db_path:
                try:
                    if not os.path.exists('/content/drive'):
                        from google.colab import drive
                        drive.mount('/content/drive')
                        logger.info("Google Drive mounted successfully")
                except Exception as e:
                    logger.warning(f"Google Drive mounting failed: {e}")
                    # Fallback to local path
                    fallback_path = "/content/chatbot_memory.db"
                    logger.info(f"Using local database path: {fallback_path}")
                    return fallback_path

            # Create directory if it doesn't exist
            os.makedirs(db_dir, exist_ok=True)

            # Test if we can write to the path
            test_file = db_path + ".test"
            try:
                with open(test_file, 'w') as f:
                    f.write("test")
                os.remove(test_file)
                return db_path
            except Exception as e:
                logger.warning(f"Cannot write to {db_path}: {e}")
                # Fallback to local path
                fallback_path = "/content/chatbot_memory.db"
                logger.info(f"Using local database path: {fallback_path}")
                return fallback_path

        except Exception as e:
            logger.warning(f"Database path setup failed: {e}")
            # Final fallback
            fallback_path = "/content/chatbot_memory.db"
            logger.info(f"Using local database path: {fallback_path}")
            return fallback_path

    def init_database(self):
        """Initialize SQLite database with required tables"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()

                # Users table
                cursor.execute('''
                CREATE TABLE IF NOT EXISTS users (
                    user_id TEXT PRIMARY KEY,
                    name TEXT NOT NULL,
                    preferred_personality TEXT,
                    conversation_style TEXT,
                    topics_of_interest TEXT,  -- JSON string
                    sentiment_history TEXT,   -- JSON string
                    personality_preferences TEXT,  -- JSON string
                    learning_goals TEXT,      -- JSON string
                    interaction_patterns TEXT,  -- JSON string
                    created_at TEXT,
                    last_active TEXT
                )
                ''')

                # Conversations table
                cursor.execute('''
                CREATE TABLE IF NOT EXISTS conversations (
                    conversation_id TEXT PRIMARY KEY,
                    user_id TEXT,
                    summary TEXT,
                    key_topics TEXT,  -- JSON string
                    overall_sentiment REAL,
                    personality_evolution TEXT,  -- JSON string
                    importance_score REAL,
                    created_at TEXT,
                    updated_at TEXT,
                    FOREIGN KEY (user_id) REFERENCES users (user_id)
                )
                ''')

                # Messages table
                cursor.execute('''
                CREATE TABLE IF NOT EXISTS messages (
                    message_id TEXT PRIMARY KEY,
                    conversation_id TEXT,
                    user_id TEXT,
                    content TEXT NOT NULL,
                    timestamp TEXT,
                    message_type TEXT,  -- 'user' or 'assistant'
                    sentiment_score REAL,
                    emotion TEXT,
                    personality_used TEXT,
                    context_sources TEXT,  -- JSON string
                    metadata TEXT,  -- JSON string
                    FOREIGN KEY (conversation_id) REFERENCES conversations (conversation_id),
                    FOREIGN KEY (user_id) REFERENCES users (user_id)
                )
                ''')

                # Knowledge base table for RAG
                cursor.execute('''
                CREATE TABLE IF NOT EXISTS knowledge_chunks (
                    chunk_id TEXT PRIMARY KEY,
                    source_document TEXT,
                    content TEXT NOT NULL,
                    embedding_vector BLOB,  -- Pickled numpy array
                    chunk_index INTEGER,
                    metadata TEXT,  -- JSON string
                    created_at TEXT
                )
                ''')

                conn.commit()
                logger.info("Database initialized successfully")

        except Exception as e:
            logger.error(f"Database initialization failed: {e}")
            raise

    def save_user_profile(self, profile: UserProfile):
        """Save or update user profile"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                cursor.execute('''
                INSERT OR REPLACE INTO users
                (user_id, name, preferred_personality, conversation_style,
                 topics_of_interest, sentiment_history, personality_preferences,
                 learning_goals, interaction_patterns, created_at, last_active)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    profile.user_id, profile.name, profile.preferred_personality,
                    profile.conversation_style, json.dumps(profile.topics_of_interest),
                    json.dumps(profile.sentiment_history),
                    json.dumps(profile.personality_preferences),
                    json.dumps(profile.learning_goals),
                    json.dumps(profile.interaction_patterns),
                    profile.created_at.isoformat(), profile.last_active.isoformat()
                ))
                conn.commit()
                logger.info(f"User profile saved: {profile.user_id}")
        except Exception as e:
            logger.error(f"Failed to save user profile: {e}")

    def load_user_profile(self, user_id: str) -> Optional[UserProfile]:
        """Load user profile from database"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                cursor.execute('SELECT * FROM users WHERE user_id = ?', (user_id,))
                row = cursor.fetchone()

                if row:
                    return UserProfile(
                        user_id=row[0], name=row[1], preferred_personality=row[2],
                        conversation_style=row[3], topics_of_interest=json.loads(row[4]),
                        sentiment_history=json.loads(row[5]),
                        personality_preferences=json.loads(row[6]),
                        learning_goals=json.loads(row[7]),
                        interaction_patterns=json.loads(row[8]),
                        created_at=datetime.fromisoformat(row[9]),
                        last_active=datetime.fromisoformat(row[10])
                    )
                return None
        except Exception as e:
            logger.error(f"Failed to load user profile: {e}")
            return None

    def save_message(self, message: Message, conversation_id: str):
        """Save a message to the database"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                cursor.execute('''
                INSERT INTO messages
                (message_id, conversation_id, user_id, content, timestamp,
                 message_type, sentiment_score, emotion, personality_used,
                 context_sources, metadata)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    message.id, conversation_id, message.user_id, message.content,
                    message.timestamp.isoformat(), message.message_type,
                    message.sentiment_score, message.emotion, message.personality_used,
                    json.dumps(message.context_sources), json.dumps(message.metadata)
                ))
                conn.commit()
                logger.debug(f"Message saved: {message.id}")
        except Exception as e:
            logger.error(f"Failed to save message: {e}")

    def get_conversation_history(self, user_id: str, limit: int = 50) -> List[Message]:
        """Retrieve recent conversation history for a user"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                cursor.execute('''
                SELECT * FROM messages
                WHERE user_id = ?
                ORDER BY timestamp DESC
                LIMIT ?
                ''', (user_id, limit))
                rows = cursor.fetchall()

                messages = []
                for row in rows:
                    message = Message(
                        id=row[0], user_id=row[2], content=row[3],
                        timestamp=datetime.fromisoformat(row[4]),
                        message_type=row[5], sentiment_score=row[6],
                        emotion=row[7], personality_used=row[8],
                        context_sources=json.loads(row[9]) if row[9] else [],
                        metadata=json.loads(row[10]) if row[10] else {}
                    )
                    messages.append(message)

                return list(reversed(messages))  # Return in chronological order
        except Exception as e:
            logger.error(f"Failed to get conversation history: {e}")
            return []

In [15]:
class AdvancedMemorySystem:
    """Sophisticated memory system with semantic search and context management"""

    def __init__(self, config: ChatbotConfig):
        self.config = config
        self.db_manager = DatabaseManager(config.MEMORY_DB_PATH)
        self.embedding_model = None
        self.conversation_cache = {}  # In-memory cache for active conversations
        self.user_context_cache = {}  # Cache user context for quick access

        # Initialize vector database for semantic search
        self.vector_db = None
        self._init_vector_database()

    def _init_vector_database(self):
        """Initialize ChromaDB for semantic memory search"""
        try:
            # Create ChromaDB client
            self.vector_db = chromadb.Client(Settings(
                persist_directory="/content/drive/MyDrive/chromadb",
                is_persistent=True
            ))

            # Get or create collection for conversation memories
            self.memory_collection = self.vector_db.get_or_create_collection(
                name="conversation_memories",
                metadata={"description": "Semantic search for conversation history"}
            )

            logger.info("Vector database initialized successfully")
        except Exception as e:
            logger.error(f"Vector database initialization failed: {e}")

    def load_embedding_model(self):
        """Load sentence transformer model for embeddings"""
        if self.embedding_model is None:
            try:
                self.embedding_model = SentenceTransformer(self.config.EMBEDDING_MODEL)
                logger.info(f"Embedding model loaded: {self.config.EMBEDDING_MODEL}")
            except Exception as e:
                logger.error(f"Failed to load embedding model: {e}")
                raise

    def create_user_profile(self, user_id: str, name: str) -> UserProfile:
        """Create a new user profile with default settings"""
        profile = UserProfile(
            user_id=user_id,
            name=name,
            preferred_personality="friendly",
            conversation_style="casual",
            topics_of_interest=[],
            sentiment_history=[],
            personality_preferences={p: 0.5 for p in DEFAULT_PERSONALITIES.keys()},
            learning_goals=[],
            interaction_patterns={
                "message_count": 0,
                "avg_response_length": 0,
                "preferred_time": None,
                "question_types": []
            },
            created_at=datetime.now(),
            last_active=datetime.now()
        )

        self.db_manager.save_user_profile(profile)
        logger.info(f"New user profile created: {user_id}")
        return profile

    def get_or_create_user(self, user_id: str, name: str = None) -> UserProfile:
        """Get existing user or create new one"""
        profile = self.db_manager.load_user_profile(user_id)
        if profile is None:
            if name is None:
                name = f"User_{user_id[:8]}"
            profile = self.create_user_profile(user_id, name)
        else:
            # Update last active time
            profile.last_active = datetime.now()
            self.db_manager.save_user_profile(profile)

        return profile

    def store_message(self, message: Message, conversation_id: str = None):
        """Store message with semantic indexing"""
        if conversation_id is None:
            conversation_id = f"conv_{message.user_id}_{datetime.now().strftime('%Y%m%d')}"

        # Save to database
        self.db_manager.save_message(message, conversation_id)

        # Add to semantic search index
        self._add_to_semantic_index(message, conversation_id)

        # Update user interaction patterns
        self._update_user_patterns(message)

    def _add_to_semantic_index(self, message: Message, conversation_id: str):
        """Add message to vector database for semantic search"""
        if self.vector_db is None or self.embedding_model is None:
            return

        try:
            # Create embedding
            embedding = self.embedding_model.encode([message.content])[0]

            # Prepare metadata
            metadata = {
                "user_id": message.user_id,
                "conversation_id": conversation_id,
                "timestamp": message.timestamp.isoformat(),
                "message_type": message.message_type,
                "emotion": message.emotion,
                "sentiment_score": message.sentiment_score,
                "personality_used": message.personality_used
            }

            # Add to collection
            self.memory_collection.add(
                embeddings=[embedding.tolist()],
                documents=[message.content],
                metadatas=[metadata],
                ids=[message.id]
            )

            logger.debug(f"Message added to semantic index: {message.id}")
        except Exception as e:
            logger.error(f"Failed to add message to semantic index: {e}")

    def _update_user_patterns(self, message: Message):
        """Update user interaction patterns based on new message"""
        try:
            profile = self.db_manager.load_user_profile(message.user_id)
            if profile:
                # Update message count
                profile.interaction_patterns["message_count"] += 1

                # Update average response length
                if message.message_type == "user":
                    current_avg = profile.interaction_patterns.get("avg_response_length", 0)
                    count = profile.interaction_patterns["message_count"]
                    new_avg = (current_avg * (count - 1) + len(message.content)) / count
                    profile.interaction_patterns["avg_response_length"] = new_avg

                # Update sentiment history (keep last 100)
                profile.sentiment_history.append(message.sentiment_score)
                if len(profile.sentiment_history) > 100:
                    profile.sentiment_history = profile.sentiment_history[-100:]

                # Save updated profile
                self.db_manager.save_user_profile(profile)
        except Exception as e:
            logger.error(f"Failed to update user patterns: {e}")

    def retrieve_relevant_memories(self, query: str, user_id: str, limit: int = 5) -> List[Dict]:
        """Retrieve semantically similar memories for context"""
        if self.vector_db is None or self.embedding_model is None:
            return []

        try:
            # Create query embedding
            query_embedding = self.embedding_model.encode([query])[0]

            # Search for similar memories
            results = self.memory_collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=limit,
                where={"user_id": user_id}
            )

            relevant_memories = []
            if results['documents'] and results['documents'][0]:
                for i, doc in enumerate(results['documents'][0]):
                    memory = {
                        "content": doc,
                        "metadata": results['metadatas'][0][i],
                        "similarity": 1 - results['distances'][0][i]  # Convert distance to similarity
                    }
                    # Only include memories above similarity threshold
                    if memory["similarity"] >= self.config.SIMILARITY_THRESHOLD:
                        relevant_memories.append(memory)

            logger.debug(f"Retrieved {len(relevant_memories)} relevant memories for query")
            return relevant_memories

        except Exception as e:
            logger.error(f"Failed to retrieve relevant memories: {e}")
            return []

    def get_conversation_context(self, user_id: str, current_message: str) -> Dict[str, Any]:
        """Get comprehensive conversation context for response generation"""
        context = {
            "user_profile": self.get_or_create_user(user_id),
            "recent_history": self.db_manager.get_conversation_history(user_id, limit=10),
            "relevant_memories": self.retrieve_relevant_memories(current_message, user_id),
            "sentiment_trend": self._analyze_sentiment_trend(user_id),
            "conversation_summary": self._generate_conversation_summary(user_id)
        }

        return context

    def _analyze_sentiment_trend(self, user_id: str) -> Dict[str, float]:
        """Analyze recent sentiment trends for the user"""
        try:
            profile = self.db_manager.load_user_profile(user_id)
            if profile and profile.sentiment_history:
                recent_sentiments = profile.sentiment_history[-10:]  # Last 10 messages
                return {
                    "current_sentiment": recent_sentiments[-1] if recent_sentiments else 0.0,
                    "average_sentiment": np.mean(recent_sentiments),
                    "sentiment_trend": np.mean(recent_sentiments[-5:]) - np.mean(recent_sentiments[-10:-5]) if len(recent_sentiments) >= 10 else 0.0
                }
        except Exception as e:
            logger.error(f"Failed to analyze sentiment trend: {e}")

        return {"current_sentiment": 0.0, "average_sentiment": 0.0, "sentiment_trend": 0.0}

    def _generate_conversation_summary(self, user_id: str) -> str:
        """Generate a brief summary of recent conversation topics"""
        try:
            recent_messages = self.db_manager.get_conversation_history(user_id, limit=20)
            if not recent_messages:
                return "No previous conversation history."

            # Extract user messages for topic analysis
            user_messages = [msg.content for msg in recent_messages if msg.message_type == "user"]

            if user_messages:
                # Simple keyword extraction for topics using NLTK stopwords
                all_text = " ".join(user_messages[-10:])  # Last 10 user messages
                words = word_tokenize(all_text.lower())

                # Filter out stopwords and short words
                stop_words = set(stopwords.words('english'))
                filtered_words = [w for w in words if w.isalpha() and w not in stop_words and len(w) > 3]

                if filtered_words:
                    word_freq = {}
                    for word in filtered_words:
                        word_freq[word] = word_freq.get(word, 0) + 1

                    top_topics = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:5]
                    topics = [topic[0] for topic in top_topics]
                    return f"Recent topics: {', '.join(topics)}"

            return "General conversation"

        except Exception as e:
            logger.error(f"Failed to generate conversation summary: {e}")
            return "Unable to summarize conversation"

# ***RAG System Implementation***

In [16]:
class DocumentProcessor:
    """Handles document processing, chunking, and embedding for RAG system"""

    def __init__(self, config: ChatbotConfig):
        self.config = config
        self.embedding_model = None

    def load_embedding_model(self):
        """Load the sentence transformer model for embeddings"""
        if self.embedding_model is None:
            self.embedding_model = SentenceTransformer(self.config.EMBEDDING_MODEL)
            logger.info(f"Document embedding model loaded: {self.config.EMBEDDING_MODEL}")

    def chunk_text(self, text: str, chunk_size: int = None, overlap: int = None) -> List[str]:
        """Split text into overlapping chunks for better context preservation"""
        chunk_size = chunk_size or self.config.CHUNK_SIZE
        overlap = overlap or self.config.CHUNK_OVERLAP

        #Split by sentences first for better semantic boundaries
        sentences = nltk.sent_tokenize(text)

        chunks = []
        current_chunk = ""
        current_size = 0

        for sentence in sentences:
            sentence_size = len(sentence)

            #If adding this sentence would exceed chunk size, save current chunk
            if current_size + sentence_size > chunk_size and current_chunk:
                chunks.append(current_chunk.strip())

                #Start new chunk with overlap from previous chunk
                if overlap > 0 and chunks:
                    overlap_text = current_chunk[-overlap:] if len(current_chunk) > overlap else current_chunk
                    current_chunk = overlap_text + " " + sentence
                    current_size = len(current_chunk)
                else:
                    current_chunk = sentence
                    current_size = sentence_size
            else:
                #Add sentence to current chunk
                if current_chunk:
                    current_chunk += " " + sentence
                else:
                    current_chunk = sentence
                current_size += sentence_size

        #Adding the last chunk if it exists
        if current_chunk.strip():
            chunks.append(current_chunk.strip())

        return chunks

    def process_document(self, content: str, source_name: str, metadata: Dict = None) -> List[Dict]:
        """Process a document into chunks with embeddings"""
        if not self.embedding_model:
            self.load_embedding_model()

        #Cleaning the content
        content = self.clean_text(content)

        #Creating chunks
        chunks = self.chunk_text(content)

        #Generating embeddings for each chunk
        embeddings = self.embedding_model.encode(chunks)

        #Creating chunk objects with metadata
        processed_chunks = []
        for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
            chunk_data = {
                'chunk_id': f"{source_name}_{i}",
                'source_document': source_name,
                'content': chunk,
                'embedding': embedding,
                'chunk_index': i,
                'metadata': metadata or {},
                'created_at': datetime.now().isoformat()
            }
            processed_chunks.append(chunk_data)

        logger.info(f"Processed document '{source_name}' into {len(processed_chunks)} chunks")
        return processed_chunks

    def clean_text(self, text: str) -> str:
        """Clean and normalize text content"""
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text)

        # Remove special characters but keep punctuation
        text = re.sub(r'[^\w\s\.\,\!\?\;\:\-\(\)\[\]\"\']+', '', text)

        # Normalize quotes
        text = text.replace('"', '"').replace('"', '"')
        text = text.replace(''', "'").replace(''', "'")

        return text.strip()

    def extract_text_from_file(self, file_path: str) -> str:
        """Extract text from various file formats"""
        file_extension = Path(file_path).suffix.lower()

        try:
            if file_extension == '.txt':
                with open(file_path, 'r', encoding='utf-8') as file:
                    return file.read()

            elif file_extension == '.pdf':
                # Simple PDF text extraction (would need PyPDF2 or similar in real implementation)
                logger.warning("PDF processing not implemented in this demo")
                return "PDF content would be extracted here"

            elif file_extension in ['.docx', '.doc']:
                # Word document processing (would need python-docx)
                logger.warning("Word document processing not implemented in this demo")
                return "Word document content would be extracted here"

            else:
                logger.warning(f"Unsupported file format: {file_extension}")
                return ""

        except Exception as e:
            logger.error(f"Failed to extract text from {file_path}: {e}")
            return ""

In [17]:
class KnowledgeBase:
    """Manages the knowledge base with FAISS vector search"""

    def __init__(self, config: ChatbotConfig):
        self.config = config
        self.db_manager = DatabaseManager(config.MEMORY_DB_PATH)
        self.document_processor = DocumentProcessor(config)

        # FAISS index for fast similarity search
        self.faiss_index = None
        self.chunk_metadata = []  # Store metadata for each chunk
        self.is_loaded = False

    def initialize_knowledge_base(self):
        """Initialize or load existing knowledge base"""
        try:
            self.document_processor.load_embedding_model()
            self._load_or_create_faiss_index()
            self.is_loaded = True
            logger.info("Knowledge base initialized successfully")
        except Exception as e:
            logger.error(f"Failed to initialize knowledge base: {e}")
            raise

    def _load_or_create_faiss_index(self):
        """Load existing FAISS index or create new one"""
        index_path = "/content/drive/MyDrive/faiss_index.pkl"
        metadata_path = "/content/drive/MyDrive/chunk_metadata.pkl"

        try:
            # Try to load existing index
            if os.path.exists(index_path) and os.path.exists(metadata_path):
                with open(index_path, 'rb') as f:
                    self.faiss_index = pickle.load(f)
                with open(metadata_path, 'rb') as f:
                    self.chunk_metadata = pickle.load(f)
                logger.info("Loaded existing FAISS index")
            else:
                # Create new index
                self.faiss_index = faiss.IndexFlatIP(self.config.EMBEDDING_DIMENSION)
                self.chunk_metadata = []
                logger.info("Created new FAISS index")

        except Exception as e:
            logger.error(f"Failed to load FAISS index: {e}")
            # Create new index as fallback
            self.faiss_index = faiss.IndexFlatIP(self.config.EMBEDDING_DIMENSION)
            self.chunk_metadata = []

    def add_document(self, content: str, source_name: str, metadata: Dict = None):
        """Add a document to the knowledge base"""
        if not self.is_loaded:
            self.initialize_knowledge_base()

        try:
            # Process document into chunks
            chunks = self.document_processor.process_document(content, source_name, metadata)

            # Add chunks to database
            for chunk in chunks:
                self._save_chunk_to_db(chunk)

            # Add to FAISS index
            embeddings = np.array([chunk['embedding'] for chunk in chunks]).astype('float32')

            # Normalize embeddings for cosine similarity
            faiss.normalize_L2(embeddings)

            # Add to index
            self.faiss_index.add(embeddings)

            # Store metadata
            for chunk in chunks:
                self.chunk_metadata.append({
                    'chunk_id': chunk['chunk_id'],
                    'source_document': chunk['source_document'],
                    'content': chunk['content'],
                    'chunk_index': chunk['chunk_index'],
                    'metadata': chunk['metadata']
                })

            # Save updated index
            self._save_faiss_index()

            logger.info(f"Added document '{source_name}' with {len(chunks)} chunks to knowledge base")

        except Exception as e:
            logger.error(f"Failed to add document to knowledge base: {e}")

    def _save_chunk_to_db(self, chunk: Dict):
        """Save chunk to SQLite database"""
        try:
            with sqlite3.connect(self.config.MEMORY_DB_PATH) as conn:
                cursor = conn.cursor()
                cursor.execute('''
                INSERT OR REPLACE INTO knowledge_chunks
                (chunk_id, source_document, content, embedding_vector,
                 chunk_index, metadata, created_at)
                VALUES (?, ?, ?, ?, ?, ?, ?)
                ''', (
                    chunk['chunk_id'],
                    chunk['source_document'],
                    chunk['content'],
                    pickle.dumps(chunk['embedding']),
                    chunk['chunk_index'],
                    json.dumps(chunk['metadata']),
                    chunk['created_at']
                ))
                conn.commit()
        except Exception as e:
            logger.error(f"Failed to save chunk to database: {e}")

    def _save_faiss_index(self):
        """Save FAISS index and metadata to disk"""
        try:
            os.makedirs("/content/drive/MyDrive", exist_ok=True)

            with open("/content/drive/MyDrive/faiss_index.pkl", 'wb') as f:
                pickle.dump(self.faiss_index, f)

            with open("/content/drive/MyDrive/chunk_metadata.pkl", 'wb') as f:
                pickle.dump(self.chunk_metadata, f)

            logger.debug("FAISS index saved successfully")
        except Exception as e:
            logger.error(f"Failed to save FAISS index: {e}")

    def search_knowledge(self, query: str, top_k: int = None) -> List[Dict]:
        """Search knowledge base for relevant information"""
        if not self.is_loaded or self.faiss_index.ntotal == 0:
            return []

        top_k = top_k or self.config.TOP_K_RETRIEVAL

        try:
            # Generate query embedding
            query_embedding = self.document_processor.embedding_model.encode([query]).astype('float32')
            faiss.normalize_L2(query_embedding)

            # Search FAISS index
            similarities, indices = self.faiss_index.search(query_embedding, top_k)

            # Prepare results
            results = []
            for i, (similarity, idx) in enumerate(zip(similarities[0], indices[0])):
                if idx < len(self.chunk_metadata) and similarity > 0.1:  # Minimum similarity threshold
                    chunk_info = self.chunk_metadata[idx].copy()
                    chunk_info['similarity_score'] = float(similarity)
                    chunk_info['rank'] = i + 1
                    results.append(chunk_info)

            logger.debug(f"Found {len(results)} relevant chunks for query")
            return results

        except Exception as e:
            logger.error(f"Failed to search knowledge base: {e}")
            return []

    def get_knowledge_stats(self) -> Dict[str, Any]:
        """Get statistics about the knowledge base"""
        stats = {
            'total_chunks': self.faiss_index.ntotal if self.faiss_index else 0,
            'total_documents': len(set(chunk['source_document'] for chunk in self.chunk_metadata)),
            'embedding_dimension': self.config.EMBEDDING_DIMENSION,
            'is_loaded': self.is_loaded
        }

        if self.chunk_metadata:
            # Document distribution
            doc_counts = {}
            for chunk in self.chunk_metadata:
                doc = chunk['source_document']
                doc_counts[doc] = doc_counts.get(doc, 0) + 1
            stats['document_distribution'] = doc_counts

        return stats

In [18]:
class RAGSystem:
    """Complete RAG system that combines retrieval and generation"""

    def __init__(self, config: ChatbotConfig):
        self.config = config
        self.knowledge_base = KnowledgeBase(config)
        self.memory_system = None  # Will be injected from main system

    def initialize(self):
        """Initialize the RAG system"""
        self.knowledge_base.initialize_knowledge_base()
        logger.info("RAG system initialized successfully")

    def create_sample_knowledge_base(self):
        """Create a sample knowledge base for demonstration"""
        sample_documents = {
            "ai_basics": """
            Artificial Intelligence (AI) is a branch of computer science that aims to create
            intelligent machines that work and react like humans. Machine Learning is a subset
            of AI that provides systems the ability to automatically learn and improve from
            experience without being explicitly programmed. Deep Learning is a subset of machine
            learning that uses neural networks with multiple layers to model and understand
            complex patterns in data.

            Natural Language Processing (NLP) is a field of AI that focuses on the interaction
            between computers and humans through natural language. It involves programming
            computers to process and analyze large amounts of natural language data.
            """,

            "python_programming": """
            Python is a high-level, interpreted programming language with dynamic semantics.
            Its high-level built-in data structures, combined with dynamic typing and dynamic
            binding, make it very attractive for Rapid Application Development. Python supports
            modules and packages, which encourages program modularity and code reuse.

            Key Python concepts include: variables and data types, control structures like
            loops and conditionals, functions and classes, exception handling, and file I/O.
            Popular Python libraries include NumPy for numerical computing, Pandas for data
            analysis, Matplotlib for plotting, and Scikit-learn for machine learning.
            """,

            "data_science": """
            Data Science is an interdisciplinary field that uses scientific methods, processes,
            algorithms and systems to extract knowledge and insights from structured and
            unstructured data. It combines domain expertise, programming skills, and knowledge
            of mathematics and statistics to extract meaningful insights from data.

            The data science process typically includes: data collection, data cleaning,
            exploratory data analysis, feature engineering, model building, model evaluation,
            and deployment. Common tools include Python, R, SQL, Jupyter notebooks, and
            various cloud platforms.
            """
        }

        logger.info("Creating sample knowledge base...")
        for doc_name, content in sample_documents.items():
            self.knowledge_base.add_document(content, doc_name, {"type": "educational", "topic": doc_name})

        logger.info("Sample knowledge base created successfully")

    def retrieve_and_format_context(self, query: str, user_id: str = None) -> str:
        """Retrieve relevant knowledge and format it for context injection"""
        # Search knowledge base
        knowledge_results = self.knowledge_base.search_knowledge(query)

        # Get conversation memory if user_id provided
        memory_context = ""
        if user_id and self.memory_system:
            relevant_memories = self.memory_system.retrieve_relevant_memories(query, user_id, limit=3)
            if relevant_memories:
                memory_parts = []
                for memory in relevant_memories:
                    memory_parts.append(f"Previous conversation: {memory['content']}")
                memory_context = "\n".join(memory_parts)

        # Format knowledge context
        knowledge_context = ""
        if knowledge_results:
            knowledge_parts = []
            for result in knowledge_results:
                source = result['source_document']
                content = result['content']
                score = result['similarity_score']
                knowledge_parts.append(f"From {source} (relevance: {score:.2f}):\n{content}")
            knowledge_context = "\n\n".join(knowledge_parts)

        # Combine contexts
        full_context = []
        if knowledge_context:
            full_context.append(f"Relevant Knowledge:\n{knowledge_context}")
        if memory_context:
            full_context.append(f"Conversation Memory:\n{memory_context}")

        return "\n\n" + "="*50 + "\n" + "\n\n".join(full_context) + "\n" + "="*50 if full_context else ""

    def get_rag_stats(self) -> Dict[str, Any]:
        """Get comprehensive RAG system statistics"""
        return {
            "knowledge_base": self.knowledge_base.get_knowledge_stats(),
            "retrieval_config": {
                "top_k": self.config.TOP_K_RETRIEVAL,
                "chunk_size": self.config.CHUNK_SIZE,
                "chunk_overlap": self.config.CHUNK_OVERLAP,
                "embedding_model": self.config.EMBEDDING_MODEL
            }
        }

# ***Sentiment Analysis System***

In [19]:
class SentimentAnalyzer:
    """Advanced sentiment analysis with transformer-based emotion detection"""

    def __init__(self, config: ChatbotConfig):
        self.config = config
        self.sentiment_pipeline = None
        self.emotion_pipeline = None
        self.sentiment_mapping = {
            'LABEL_0': 'negative',
            'LABEL_1': 'neutral',
            'LABEL_2': 'positive',
            'NEGATIVE': 'negative',
            'NEUTRAL': 'neutral',
            'POSITIVE': 'positive'
        }

        # Emotion model for sophisticated emotion detection
        self.emotion_model_name = config.EMOTION_MODEL

        # Fallback patterns only used if models fail to load
        self.fallback_patterns = {
            'joy': ['happy', 'excited', 'great', 'amazing', 'wonderful'],
            'sadness': ['sad', 'disappointed', 'upset', 'terrible'],
            'anger': ['angry', 'frustrated', 'annoyed', 'mad'],
            'fear': ['scared', 'worried', 'anxious', 'afraid'],
            'surprise': ['surprised', 'shocked', 'unexpected', 'wow'],
            'neutral': ['okay', 'fine', 'normal', 'regular']
        }

    def load_models(self):
        """Load both sentiment and emotion analysis models"""
        if self.sentiment_pipeline is None:
            try:
                self.sentiment_pipeline = pipeline(
                    "sentiment-analysis",
                    model=self.config.SENTIMENT_MODEL,
                    device=0 if torch.cuda.is_available() else -1
                )
                logger.info(f"Sentiment model loaded: {self.config.SENTIMENT_MODEL}")
            except Exception as e:
                logger.warning(f"Failed to load sentiment model: {e}")

        if self.emotion_pipeline is None:
            try:
                self.emotion_pipeline = pipeline(
                    "text-classification",
                    model=self.emotion_model_name,
                    device=0 if torch.cuda.is_available() else -1
                )
                logger.info(f"Emotion model loaded: {self.emotion_model_name}")
            except Exception as e:
                logger.warning(f"Failed to load emotion model: {e}")

    def analyze_sentiment(self, text: str) -> Dict[str, Any]:
        """Perform comprehensive sentiment and emotion analysis"""
        if not text.strip():
            return self._empty_sentiment_result()

        # Ensure models are loaded
        self.load_models()

        try:
            # Primary sentiment analysis
            sentiment_score, confidence = self._get_sentiment_score(text)

            # Advanced emotion detection
            emotion = self._detect_emotion_advanced(text)

            # Text statistics
            text_stats = self._calculate_text_statistics(text)

            # Intensity analysis
            intensity = self._analyze_intensity(text)

            result = {
                'sentiment_score': sentiment_score,
                'sentiment_label': self._score_to_label(sentiment_score),
                'confidence': confidence,
                'emotion': emotion,
                'intensity': intensity,
                'text_statistics': text_stats,
                'timestamp': datetime.now().isoformat()
            }

            logger.debug(f"Analysis: {result['sentiment_label']} ({sentiment_score:.3f}), emotion: {emotion}")
            return result

        except Exception as e:
            logger.error(f"Sentiment analysis failed: {e}")
            return self._empty_sentiment_result()

    def _get_sentiment_score(self, text: str) -> Tuple[float, float]:
        """Get primary sentiment score and confidence"""
        if self.sentiment_pipeline:
            try:
                # Use transformer model
                result = self.sentiment_pipeline(text)[0]
                label = result['label']
                score = result['score']

                # Convert to normalized sentiment score (-1 to 1)
                if label in ['NEGATIVE', 'LABEL_0']:
                    sentiment_score = -score
                elif label in ['POSITIVE', 'LABEL_2']:
                    sentiment_score = score
                else:  # NEUTRAL, LABEL_1
                    sentiment_score = 0.0

                return sentiment_score, score

            except Exception as e:
                logger.warning(f"Transformer sentiment analysis failed: {e}")

        # Fallback to TextBlob
        blob = TextBlob(text)
        polarity = blob.sentiment.polarity  # Range: -1 to 1
        confidence = abs(polarity) if polarity != 0 else 0.1

        return polarity, confidence

    def _detect_emotion_advanced(self, text: str) -> str:
        """Advanced emotion detection using transformer model"""
        if self.emotion_pipeline:
            try:
                # Use transformer model for emotion detection
                emotion_result = self.emotion_pipeline(text)[0]
                emotion_label = emotion_result['label'].lower()
                confidence = emotion_result['score']

                # Only use result if confidence is reasonable
                if confidence > 0.3:
                    logger.debug(f"Emotion detected: {emotion_label} (confidence: {confidence:.3f})")
                    return emotion_label

            except Exception as e:
                logger.warning(f"Transformer emotion detection failed: {e}")

        # Fallback to pattern-based detection with negation handling
        return self._detect_emotion_fallback(text)

    def _detect_emotion_fallback(self, text: str) -> str:
        """Fallback emotion detection with simple negation handling"""
        text_lower = text.lower()

        # Simple negation detection
        negation_words = ['not', 'no', 'never', 'dont', "don't", 'cant', "can't", 'wont', "won't"]
        has_negation = any(neg in text_lower for neg in negation_words)

        emotion_scores = {}

        # Score each emotion based on keyword matches
        for emotion, keywords in self.fallback_patterns.items():
            score = 0
            for keyword in keywords:
                if keyword in text_lower:
                    # Check for word boundaries to avoid partial matches
                    if f" {keyword} " in f" {text_lower} " or text_lower.startswith(keyword) or text_lower.endswith(keyword):
                        score += 1
            emotion_scores[emotion] = score

        # Find highest scoring emotion
        if emotion_scores and max(emotion_scores.values()) > 0:
            detected_emotion = max(emotion_scores, key=emotion_scores.get)

            # Apply simple negation logic
            if has_negation:
                if detected_emotion == 'joy':
                    return 'sadness'
                elif detected_emotion == 'sadness':
                    return 'joy'
                # For other emotions, negation makes them neutral
                elif detected_emotion not in ['neutral']:
                    return 'neutral'

            return detected_emotion

        # Final fallback based on sentiment
        sentiment_score, _ = self._get_sentiment_score(text)
        if sentiment_score > 0.3:
            return 'joy'
        elif sentiment_score < -0.3:
            return 'sadness'
        else:
            return 'neutral'

    def _analyze_intensity(self, text: str) -> float:
        """Analyze the intensity/strength of emotional expression"""
        # Factors that increase intensity
        intensity_factors = 0.0

        # Punctuation intensity
        exclamation_count = text.count('!')
        question_count = text.count('?')
        intensity_factors += min(exclamation_count * 0.2, 1.0)
        intensity_factors += min(question_count * 0.1, 0.5)

        # Capitalization intensity
        caps_ratio = sum(1 for c in text if c.isupper()) / max(len(text), 1)
        intensity_factors += min(caps_ratio * 2, 1.0)

        # Word intensity indicators
        intensity_words = ['very', 'really', 'extremely', 'absolutely', 'totally', 'completely', 'quite', 'so', 'too']
        for word in intensity_words:
            if word in text.lower():
                intensity_factors += 0.3

        # Normalize to 0-1 range
        return min(intensity_factors, 1.0)

    def _calculate_text_statistics(self, text: str) -> Dict[str, Any]:
        """Calculate various text statistics for analysis"""
        sentences = nltk.sent_tokenize(text)
        words = word_tokenize(text.lower())

        # Filter words (remove punctuation)
        words_only = [word for word in words if word.isalpha()]

        stats = {
            'char_count': len(text),
            'word_count': len(words_only),
            'sentence_count': len(sentences),
            'avg_word_length': np.mean([len(word) for word in words_only]) if words_only else 0,
            'avg_sentence_length': np.mean([len(nltk.word_tokenize(s)) for s in sentences]) if sentences else 0,
            'readability_score': textstat.flesch_reading_ease(text) if len(text) > 0 else 0
        }

        return stats

    def _score_to_label(self, score: float) -> str:
        """Convert sentiment score to human-readable label"""
        if score > 0.1:
            return 'positive'
        elif score < -0.1:
            return 'negative'
        else:
            return 'neutral'

    def _empty_sentiment_result(self) -> Dict[str, Any]:
        """Return empty/neutral sentiment result for error cases"""
        return {
            'sentiment_score': 0.0,
            'sentiment_label': 'neutral',
            'confidence': 0.0,
            'emotion': 'neutral',
            'intensity': 0.0,
            'text_statistics': {
                'char_count': 0, 'word_count': 0, 'sentence_count': 0,
                'avg_word_length': 0, 'avg_sentence_length': 0, 'readability_score': 0
            },
            'timestamp': datetime.now().isoformat()
        }

In [20]:
class EmotionTracker:
    """Tracks emotional patterns and trends over time"""

    def __init__(self):
        self.emotion_history = defaultdict(list)  # user_id -> list of emotion data
        self.sentiment_trends = defaultdict(deque)  # user_id -> rolling sentiment scores
        self.max_history = 100  # Keep last 100 emotional states per user

    def track_emotion(self, user_id: str, sentiment_result: Dict[str, Any]):
        """Track emotional state for a user"""
        emotion_entry = {
            'timestamp': datetime.now(),
            'sentiment_score': sentiment_result['sentiment_score'],
            'emotion': sentiment_result['emotion'],
            'intensity': sentiment_result['intensity'],
            'confidence': sentiment_result['confidence']
        }

        # Add to history
        self.emotion_history[user_id].append(emotion_entry)

        # Maintain max history size
        if len(self.emotion_history[user_id]) > self.max_history:
            self.emotion_history[user_id] = self.emotion_history[user_id][-self.max_history:]

        # Update sentiment trend (rolling window)
        self.sentiment_trends[user_id].append(sentiment_result['sentiment_score'])
        if len(self.sentiment_trends[user_id]) > 20:  # Keep last 20 scores
            self.sentiment_trends[user_id].popleft()

    def get_emotion_summary(self, user_id: str) -> Dict[str, Any]:
        """Get emotional summary for a user"""
        if user_id not in self.emotion_history:
            return {'status': 'no_data'}

        history = self.emotion_history[user_id]
        recent_history = history[-10:]  # Last 10 interactions

        # Calculate emotion distribution
        emotion_counts = defaultdict(int)
        for entry in recent_history:
            emotion_counts[entry['emotion']] += 1

        # Calculate sentiment trend
        sentiment_scores = [entry['sentiment_score'] for entry in recent_history]

        # Calculate intensity trend
        intensity_scores = [entry['intensity'] for entry in recent_history]

        summary = {
            'total_interactions': len(history),
            'recent_interactions': len(recent_history),
            'dominant_emotion': max(emotion_counts, key=emotion_counts.get) if emotion_counts else 'neutral',
            'emotion_distribution': dict(emotion_counts),
            'average_sentiment': np.mean(sentiment_scores) if sentiment_scores else 0.0,
            'sentiment_trend': self._calculate_trend(sentiment_scores),
            'average_intensity': np.mean(intensity_scores) if intensity_scores else 0.0,
            'emotional_stability': np.std(sentiment_scores) if len(sentiment_scores) > 1 else 0.0,
            'last_emotion': recent_history[-1]['emotion'] if recent_history else 'neutral',
            'last_sentiment': recent_history[-1]['sentiment_score'] if recent_history else 0.0
        }

        return summary

    def _calculate_trend(self, scores: List[float]) -> str:
        """Calculate if sentiment is improving, declining, or stable"""
        if len(scores) < 3:
            return 'stable'

        # Compare first half vs second half
        mid = len(scores) // 2
        first_half_avg = np.mean(scores[:mid])
        second_half_avg = np.mean(scores[mid:])

        diff = second_half_avg - first_half_avg

        if diff > 0.1:
            return 'improving'
        elif diff < -0.1:
            return 'declining'
        else:
            return 'stable'

    def get_mood_recommendation(self, user_id: str) -> Dict[str, str]:
        """Get personality and response recommendations based on emotional state"""
        summary = self.get_emotion_summary(user_id)

        if summary.get('status') == 'no_data':
            return {'recommended_personality': 'friendly', 'response_style': 'neutral'}

        last_emotion = summary['last_emotion']
        last_sentiment = summary['last_sentiment']
        sentiment_trend = summary['sentiment_trend']

        # Recommendation logic
        recommendations = {
            'recommended_personality': 'friendly',
            'response_style': 'supportive',
            'tone_adjustment': 'neutral'
        }

        # Adjust based on current emotion
        if last_emotion in ['sadness', 'fear', 'frustration']:
            recommendations['recommended_personality'] = 'friendly'
            recommendations['response_style'] = 'empathetic'
            recommendations['tone_adjustment'] = 'gentle'

        elif last_emotion in ['joy', 'satisfaction', 'curiosity']:
            recommendations['recommended_personality'] = 'creative'
            recommendations['response_style'] = 'enthusiastic'
            recommendations['tone_adjustment'] = 'energetic'

        elif last_emotion in ['anger', 'disgust']:
            recommendations['recommended_personality'] = 'professional'
            recommendations['response_style'] = 'calm'
            recommendations['tone_adjustment'] = 'measured'

        elif last_emotion == 'confusion':
            recommendations['recommended_personality'] = 'analytical'
            recommendations['response_style'] = 'explanatory'
            recommendations['tone_adjustment'] = 'clear'

        # Adjust based on sentiment trend
        if sentiment_trend == 'declining':
            recommendations['response_style'] = 'encouraging'
            recommendations['tone_adjustment'] = 'positive'

        return recommendations

In [21]:
class SentimentVisualization:
    """Creates visualizations for sentiment analysis data"""

    @staticmethod
    def create_sentiment_timeline(emotion_history: List[Dict], title: str = "Sentiment Timeline") -> go.Figure:
        """Create an interactive sentiment timeline"""
        if not emotion_history:
            return go.Figure().add_annotation(text="No data available", xref="paper", yref="paper", x=0.5, y=0.5)

        timestamps = [entry['timestamp'] for entry in emotion_history]
        sentiment_scores = [entry['sentiment_score'] for entry in emotion_history]
        emotions = [entry['emotion'] for entry in emotion_history]
        intensities = [entry['intensity'] for entry in emotion_history]

        fig = go.Figure()

        # Sentiment line
        fig.add_trace(go.Scatter(
            x=timestamps,
            y=sentiment_scores,
            mode='lines+markers',
            name='Sentiment Score',
            line=dict(color='blue', width=2),
            marker=dict(size=8, color=intensities, colorscale='Viridis', showscale=True,
                       colorbar=dict(title="Intensity")),
            hovertemplate="<b>%{y:.3f}</b><br>%{x}<br>Emotion: %{text}<extra></extra>",
            text=emotions
        ))

        # Add reference lines
        fig.add_hline(y=0, line_dash="dash", line_color="gray", annotation_text="Neutral")
        fig.add_hline(y=0.5, line_dash="dot", line_color="green", annotation_text="Positive")
        fig.add_hline(y=-0.5, line_dash="dot", line_color="red", annotation_text="Negative")

        fig.update_layout(
            title=title,
            xaxis_title="Time",
            yaxis_title="Sentiment Score",
            yaxis=dict(range=[-1.1, 1.1]),
            hovermode='closest',
            template="plotly_white"
        )

        return fig

    @staticmethod
    def create_emotion_distribution(emotion_counts: Dict[str, int]) -> go.Figure:
        """Create emotion distribution pie chart"""
        if not emotion_counts:
            return go.Figure().add_annotation(text="No emotion data", xref="paper", yref="paper", x=0.5, y=0.5)

        emotions = list(emotion_counts.keys())
        counts = list(emotion_counts.values())

        fig = go.Figure(data=[go.Pie(
            labels=emotions,
            values=counts,
            hole=0.3,
            hovertemplate="<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent}<extra></extra>"
        )])

        fig.update_layout(
            title="Emotion Distribution",
            template="plotly_white"
        )

        return fig

# ***Main Chatbot Engine***

In [22]:
class ModelManager:
    """Manages loading and unloading of different AI models efficiently"""

    def __init__(self, config: ChatbotConfig):
        self.config = config
        self.loaded_models = {}
        self.current_primary_model = None
        self.tokenizer = None

        # Model loading configurations with fallback
        self.quantization_config = None
        if config.USE_4BIT_QUANTIZATION:
            try:
                self.quantization_config = BitsAndBytesConfig(
                    load_in_4bit=True,
                    bnb_4bit_compute_dtype=torch.float16,
                    bnb_4bit_use_double_quant=True,
                    bnb_4bit_quant_type="nf4"
                )
            except Exception as e:
                logger.warning(f"Failed to initialize quantization config: {e}")
                logger.info("Falling back to non-quantized model loading")
                self.quantization_config = None

    def load_primary_model(self, model_name: str = None):
        """Load the primary conversation model"""
        model_name = model_name or self.config.PRIMARY_MODEL

        if model_name in self.loaded_models:
            self.current_primary_model = self.loaded_models[model_name]
            logger.info(f"Primary model already loaded: {model_name}")
            return

        try:
            logger.info(f"Loading primary model: {model_name}")

            # Load tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)

            # Add pad token if not present
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            # Load model with quantization
            self.current_primary_model = AutoModelForCausalLM.from_pretrained(
                model_name,
                quantization_config=self.quantization_config,
                device_map=self.config.DEVICE_MAP,
                torch_dtype=torch.float16 if self.config.LOAD_IN_HALF_PRECISION else torch.float32,
                trust_remote_code=True
            )

            self.loaded_models[model_name] = self.current_primary_model
            logger.info(f"Primary model loaded successfully: {model_name}")

        except Exception as e:
            logger.error(f"Failed to load primary model {model_name}: {e}")
            # Try fallback model
            if model_name != self.config.LLM_MODEL:
                logger.info("Attempting to load fallback model")
                self.load_primary_model(self.config.LLM_MODEL)
            else:
                raise RuntimeError(f"Failed to load any primary model: {e}")

    def generate_response(self, prompt: str, personality: PersonalityProfile) -> str:
        """Generate response using the current primary model"""
        if self.current_primary_model is None or self.tokenizer is None:
            self.load_primary_model()

        try:
            # Prepare inputs
            inputs = self.tokenizer.encode(prompt, return_tensors="pt", truncation=True, max_length=self.config.CONTEXT_WINDOW)

            # Move to device
            if torch.cuda.is_available():
                inputs = inputs.cuda()

            # Generate with personality-specific parameters
            generation_config = GenerationConfig(
                max_new_tokens=self.config.MAX_NEW_TOKENS,
                temperature=personality.temperature,
                top_p=personality.top_p,
                top_k=self.config.TOP_K,
                repetition_penalty=self.config.REPETITION_PENALTY,
                do_sample=self.config.DO_SAMPLE,
                pad_token_id=self.tokenizer.pad_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
                use_cache=True
            )

            # Generate response
            with torch.no_grad():
                outputs = self.current_primary_model.generate(
                    inputs,
                    generation_config=generation_config
                )

            # Decode response
            response = self.tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)

            # Clean up response
            response = self._clean_response(response)

            logger.debug(f"Generated response length: {len(response)} characters")
            return response

        except Exception as e:
            logger.error(f"Response generation failed: {e}")
            return "I apologize, but I'm having trouble generating a response right now. Please try again."

    def _clean_response(self, response: str) -> str:
        """Clean and format the generated response"""
        # Remove extra whitespace
        response = re.sub(r'\s+', ' ', response).strip()

        # Remove incomplete sentences at the end
        sentences = nltk.sent_tokenize(response)
        if len(sentences) > 1 and not sentences[-1].endswith(('.', '!', '?')):
            response = ' '.join(sentences[:-1])

        # Ensure reasonable length
        if len(response) < 10:
            return "I understand your message, but I need a bit more context to provide a helpful response."

        return response

    def get_memory_usage(self) -> Dict[str, Any]:
        """Get current GPU memory usage"""
        if torch.cuda.is_available():
            return {
                "allocated_gb": torch.cuda.memory_allocated() / 1e9,
                "reserved_gb": torch.cuda.memory_reserved() / 1e9,
                "total_gb": torch.cuda.get_device_properties(0).total_memory / 1e9,
                "loaded_models": list(self.loaded_models.keys())
            }
        return {"status": "CUDA not available"}

In [23]:
class PersonalityManager:
    """Manages personality switching and adaptation"""

    def __init__(self, personalities: Dict[str, PersonalityProfile]):
        self.personalities = personalities
        self.user_personality_history = defaultdict(list)  # Track personality usage per user

    def get_personality(self, personality_name: str) -> PersonalityProfile:
        """Get personality profile by name"""
        return self.personalities.get(personality_name, self.personalities['friendly'])

    def recommend_personality(self, user_context: Dict[str, Any]) -> str:
        """Recommend personality based on user context and emotional state"""
        # Get sentiment analysis from context
        sentiment_analysis = user_context.get('sentiment_analysis', {})
        emotion = sentiment_analysis.get('emotion', 'neutral')
        sentiment_score = sentiment_analysis.get('sentiment_score', 0.0)

        # Get user profile
        user_profile = user_context.get('user_profile')
        preferred_personality = user_profile.preferred_personality if user_profile else 'friendly'

        # Emotion-based recommendations
        emotion_personality_map = {
            'sadness': 'friendly',      # Empathetic and supportive
            'fear': 'friendly',         # Reassuring and gentle
            'anger': 'professional',    # Calm and measured
            'frustration': 'analytical', # Clear and structured
            'confusion': 'analytical',   # Explanatory and systematic
            'joy': 'creative',          # Enthusiastic and engaging
            'satisfaction': 'creative', # Positive and inspiring
            'curiosity': 'analytical',  # Informative and detailed
            'surprise': 'friendly',     # Responsive and engaging
            'neutral': preferred_personality  # Use user preference
        }

        recommended = emotion_personality_map.get(emotion, preferred_personality)

        # Override with user preference if sentiment is neutral
        if abs(sentiment_score) < 0.3:
            recommended = preferred_personality

        return recommended

    def adapt_personality_traits(self, base_personality: PersonalityProfile, user_context: Dict[str, Any]) -> PersonalityProfile:
        """Adapt personality traits based on user context"""
        # Create a copy to avoid modifying the original
        adapted_traits = base_personality.traits.copy()

        # Get sentiment info
        sentiment_analysis = user_context.get('sentiment_analysis', {})
        sentiment_score = sentiment_analysis.get('sentiment_score', 0.0)
        intensity = sentiment_analysis.get('intensity', 0.0)

        # Adapt based on sentiment
        if sentiment_score < -0.5:  # Very negative
            adapted_traits['friendliness'] = min(1.0, adapted_traits['friendliness'] + 0.2)
            adapted_traits['formality'] = max(0.0, adapted_traits['formality'] - 0.1)
        elif sentiment_score > 0.5:  # Very positive
            adapted_traits['creativity'] = min(1.0, adapted_traits['creativity'] + 0.1)
            adapted_traits['friendliness'] = min(1.0, adapted_traits['friendliness'] + 0.1)

        # Adapt based on intensity
        if intensity > 0.7:  # High intensity emotions
            adapted_traits['formality'] = max(0.0, adapted_traits['formality'] - 0.1)

        # Create adapted personality
        adapted_personality = PersonalityProfile(
            name=f"Adapted {base_personality.name}",
            description=f"Context-adapted version of {base_personality.description}",
            traits=adapted_traits,
            system_prompt=base_personality.system_prompt,
            temperature=base_personality.temperature,
            top_p=base_personality.top_p
        )

        return adapted_personality

In [24]:
class SophisticatedChatbot:
    """Main chatbot class that orchestrates all components"""

    def __init__(self, config: ChatbotConfig = None):
        self.config = config or ChatbotConfig()

        # Initialize core components
        self.model_manager = ModelManager(self.config)
        self.memory_system = AdvancedMemorySystem(self.config)
        self.rag_system = RAGSystem(self.config)
        self.sentiment_analyzer = SentimentAnalyzer(self.config)
        self.personality_manager = PersonalityManager(DEFAULT_PERSONALITIES)
        self.emotion_tracker = EmotionTracker()

        # Conversation state
        self.active_conversations = {}  # conversation_id -> conversation state
        self.initialization_complete = False

        logger.info("Sophisticated Chatbot initialized")

    def initialize(self, create_sample_kb: bool = True):
        """Initialize all components"""
        try:
            logger.info("Initializing chatbot components...")

            # Mount Google Drive for persistence
            self._ensure_drive_mounted()

            # Initialize memory system
            self.memory_system.load_embedding_model()

            # Initialize RAG system
            self.rag_system.initialize()
            self.rag_system.memory_system = self.memory_system  # Connect systems

            # Create sample knowledge base if requested
            if create_sample_kb:
                self.rag_system.create_sample_knowledge_base()

            # Load sentiment models
            self.sentiment_analyzer.load_models()

            # Load primary conversation model
            self.model_manager.load_primary_model()

            self.initialization_complete = True
            logger.info("Chatbot initialization complete!")

        except Exception as e:
            logger.error(f"Chatbot initialization failed: {e}")
            raise

    def _ensure_drive_mounted(self):
        """Ensure Google Drive is mounted for persistence"""
        try:
            if not os.path.exists('/content/drive'):
                logger.info("Mounting Google Drive...")
                from google.colab import drive
                drive.mount('/content/drive')
                logger.info("Google Drive mounted successfully")

            # Create necessary directories
            directories = [
                self.config.KNOWLEDGE_BASE_PATH,
                self.config.CONVERSATION_LOGS,
                self.config.EMBEDDINGS_PATH,
                os.path.dirname(self.config.MEMORY_DB_PATH)
            ]

            for directory in directories:
                try:
                    os.makedirs(directory, exist_ok=True)
                except Exception as e:
                    logger.warning(f"Could not create directory {directory}: {e}")

        except Exception as e:
            logger.warning(f"Google Drive setup failed: {e}")
            logger.info("Continuing with local storage only")

    def chat(self, user_message: str, user_id: str = "default_user", personality_name: str = None) -> Dict[str, Any]:
        """Main chat interface"""
        if not self.initialization_complete:
            self.initialize()

        try:
            # Analyze user message sentiment
            sentiment_result = self.sentiment_analyzer.analyze_sentiment(user_message)

            # Track emotion
            self.emotion_tracker.track_emotion(user_id, sentiment_result)

            # Get comprehensive context
            context = self.memory_system.get_conversation_context(user_id, user_message)
            context['sentiment_analysis'] = sentiment_result

            # Determine personality
            if personality_name is None:
                personality_name = self.personality_manager.recommend_personality(context)

            base_personality = self.personality_manager.get_personality(personality_name)
            adapted_personality = self.personality_manager.adapt_personality_traits(base_personality, context)

            # Get RAG context
            rag_context = self.rag_system.retrieve_and_format_context(user_message, user_id)

            # Build conversation prompt
            prompt = self._build_conversation_prompt(
                user_message, context, adapted_personality, rag_context
            )

            # Generate response
            response = self.model_manager.generate_response(prompt, adapted_personality)

            # Analyze response sentiment
            response_sentiment = self.sentiment_analyzer.analyze_sentiment(response)

            # Store conversation in memory
            user_msg = Message(
                id=str(uuid.uuid4()), user_id=user_id, content=user_message,
                timestamp=datetime.now(), message_type="user",
                sentiment_score=sentiment_result['sentiment_score'],
                emotion=sentiment_result['emotion'],
                personality_used=personality_name,
                context_sources=[], metadata=sentiment_result
            )

            assistant_msg = Message(
                id=str(uuid.uuid4()), user_id=user_id, content=response,
                timestamp=datetime.now(), message_type="assistant",
                sentiment_score=response_sentiment['sentiment_score'],
                emotion=response_sentiment['emotion'],
                personality_used=personality_name,
                context_sources=rag_context.split('\n') if rag_context else [],
                metadata=response_sentiment
            )

            self.memory_system.store_message(user_msg)
            self.memory_system.store_message(assistant_msg)

            # Prepare response data
            response_data = {
                'response': response,
                'personality_used': personality_name,
                'user_sentiment': sentiment_result,
                'response_sentiment': response_sentiment,
                'context_used': bool(rag_context),
                'memory_usage': self.model_manager.get_memory_usage(),
                'conversation_id': f"conv_{user_id}_{datetime.now().strftime('%Y%m%d')}"
            }

            logger.info(f"Chat response generated for user {user_id}")
            return response_data

        except Exception as e:
            logger.error(f"Chat processing failed: {e}")
            return {
                'response': "I apologize, but I encountered an error processing your message. Please try again.",
                'personality_used': 'friendly',
                'error': str(e)
            }

    def _build_conversation_prompt(self, user_message: str, context: Dict, personality: PersonalityProfile, rag_context: str) -> str:
        """Build comprehensive conversation prompt"""
        prompt_parts = []

        # System prompt with personality
        prompt_parts.append(personality.get_prompt_prefix())

        # Add RAG context if available
        if rag_context:
            prompt_parts.append(f"\nRelevant information:\n{rag_context}")

        # Add conversation history (last few messages)
        recent_history = context.get('recent_history', [])
        if recent_history:
            prompt_parts.append("\nRecent conversation:")
            for msg in recent_history[-3:]:  # Last 3 messages for context
                role = "Human" if msg.message_type == "user" else "Assistant"
                prompt_parts.append(f"{role}: {msg.content}")

        # Add current user message
        prompt_parts.append(f"\nHuman: {user_message}")
        prompt_parts.append("Assistant:")

        return "\n".join(prompt_parts)

    def get_system_stats(self) -> Dict[str, Any]:
        """Get comprehensive system statistics"""
        return {
            'initialization_complete': self.initialization_complete,
            'memory_usage': self.model_manager.get_memory_usage(),
            'rag_stats': self.rag_system.get_rag_stats(),
            'personalities_available': list(self.personality_manager.personalities.keys()),
            'active_conversations': len(self.active_conversations)
        }

    def add_knowledge(self, content: str, source_name: str, metadata: Dict = None):
        """Add new knowledge to the RAG system"""
        self.rag_system.knowledge_base.add_document(content, source_name, metadata)
        logger.info(f"Knowledge added: {source_name}")

    def get_user_emotion_summary(self, user_id: str) -> Dict[str, Any]:
        """Get emotional analysis summary for a user"""
        return self.emotion_tracker.get_emotion_summary(user_id)

# ***Gradio Interface***

In [25]:
class GradioInterface:
    """Gradio interface for the sophisticated chatbot"""

    def __init__(self, chatbot: SophisticatedChatbot):
        self.chatbot = chatbot
        self.conversation_history = []
        self.current_user_id = "demo_user"

        # Custom CSS for professional styling
        self.custom_css = """
        /* Professional Dark Theme */
        .gradio-container {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }

        .main-header {
            text-align: center;
            background: rgba(255, 255, 255, 0.1);
            backdrop-filter: blur(10px);
            border-radius: 20px;
            padding: 30px;
            margin: 20px 0;
            border: 1px solid rgba(255, 255, 255, 0.2);
            box-shadow: 0 8px 32px rgba(0, 0, 0, 0.1);
        }

        .main-title {
            font-size: 3em;
            font-weight: 700;
            background: linear-gradient(45deg, #ff6b6b, #4ecdc4, #45b7d1, #96ceb4);
            background-size: 300% 300%;
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            animation: gradient 4s ease infinite;
            margin: 0;
            text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
        }

        @keyframes gradient {
            0% { background-position: 0% 50%; }
            50% { background-position: 100% 50%; }
            100% { background-position: 0% 50%; }
        }

        .subtitle {
            font-size: 1.3em;
            color: rgba(255, 255, 255, 0.9);
            margin: 15px 0;
            font-weight: 300;
            letter-spacing: 1px;
        }

        .feature-badge {
            display: inline-block;
            background: rgba(255, 255, 255, 0.2);
            padding: 8px 16px;
            margin: 5px;
            border-radius: 25px;
            color: white;
            font-size: 0.9em;
            font-weight: 500;
            border: 1px solid rgba(255, 255, 255, 0.3);
            backdrop-filter: blur(5px);
        }

        /* Chat Interface Styling */
        .chat-container {
            background: rgba(255, 255, 255, 0.05);
            border-radius: 20px;
            padding: 25px;
            backdrop-filter: blur(10px);
            border: 1px solid rgba(255, 255, 255, 0.1);
            box-shadow: 0 8px 32px rgba(0, 0, 0, 0.1);
            margin: 20px 0;
        }

        .chatbot {
            background: rgba(255, 255, 255, 0.1) !important;
            border-radius: 15px !important;
            border: 1px solid rgba(255, 255, 255, 0.2) !important;
            min-height: 500px !important;
        }

        .message {
            margin: 10px 0;
            padding: 15px;
            border-radius: 15px;
            max-width: 80%;
            word-wrap: break-word;
            backdrop-filter: blur(5px);
        }

        .user-message {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            margin-left: auto;
            text-align: right;
            box-shadow: 0 4px 15px rgba(102, 126, 234, 0.3);
        }

        .bot-message {
            background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
            color: white;
            margin-right: auto;
            box-shadow: 0 4px 15px rgba(240, 147, 251, 0.3);
        }

        /* Control Panel Styling */
        .control-panel {
            background: rgba(255, 255, 255, 0.08);
            border-radius: 15px;
            padding: 20px;
            margin: 15px 0;
            border: 1px solid rgba(255, 255, 255, 0.15);
            backdrop-filter: blur(10px);
        }

        .panel-title {
            color: rgba(255, 255, 255, 0.95);
            font-size: 1.4em;
            font-weight: 600;
            margin-bottom: 15px;
            text-align: center;
        }

        /* Button Styling */
        .primary-button {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
            border: none !important;
            border-radius: 25px !important;
            padding: 12px 30px !important;
            color: white !important;
            font-weight: 600 !important;
            font-size: 1em !important;
            transition: all 0.3s ease !important;
            box-shadow: 0 4px 15px rgba(102, 126, 234, 0.3) !important;
        }

        .primary-button:hover {
            transform: translateY(-2px) !important;
            box-shadow: 0 6px 20px rgba(102, 126, 234, 0.4) !important;
        }

        /* Stats and Analytics Styling */
        .stats-container {
            background: rgba(255, 255, 255, 0.1);
            border-radius: 15px;
            padding: 20px;
            margin: 15px 0;
            border: 1px solid rgba(255, 255, 255, 0.2);
            backdrop-filter: blur(10px);
        }

        .stat-item {
            background: rgba(255, 255, 255, 0.1);
            padding: 15px;
            margin: 10px 0;
            border-radius: 10px;
            border-left: 4px solid #4ecdc4;
            color: rgba(255, 255, 255, 0.9);
        }

        .metric-value {
            font-size: 1.8em;
            font-weight: 700;
            color: #4ecdc4;
            text-shadow: 0 0 10px rgba(78, 205, 196, 0.3);
        }

        .metric-label {
            font-size: 0.9em;
            color: rgba(255, 255, 255, 0.7);
            text-transform: uppercase;
            letter-spacing: 1px;
        }

        /* Input Styling */
        .gr-textbox, .gr-dropdown {
            background: rgba(255, 255, 255, 0.1) !important;
            border: 1px solid rgba(255, 255, 255, 0.3) !important;
            border-radius: 10px !important;
            color: white !important;
            backdrop-filter: blur(5px) !important;
        }

        .gr-textbox::placeholder {
            color: rgba(255, 255, 255, 0.6) !important;
        }

        /* Animation Classes */
        .fade-in {
            animation: fadeIn 0.6s ease-in;
        }

        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(20px); }
            to { opacity: 1; transform: translateY(0); }
        }

        .pulse {
            animation: pulse 2s infinite;
        }

        @keyframes pulse {
            0% { transform: scale(1); }
            50% { transform: scale(1.05); }
            100% { transform: scale(1); }
        }

        /* Responsive Design */
        @media (max-width: 768px) {
            .main-title {
                font-size: 2em;
            }
            .subtitle {
                font-size: 1.1em;
            }
            .feature-badge {
                font-size: 0.8em;
                padding: 6px 12px;
            }
        }
        """

    def create_interface(self) -> gr.Blocks:
        """Create the main Gradio interface"""
        with gr.Blocks(
            css=self.custom_css,
            title="Sophisticated AI Chatbot - Advanced Portfolio Demo",
            theme=gr.themes.Soft(
                primary_hue="blue",
                secondary_hue="purple",
                neutral_hue="slate"
            )
        ) as demo:

            # Header Section
            with gr.Row():
                with gr.Column():
                    gr.HTML("""
                    <div class="main-header fade-in">
                        <h1 class="main-title">Sophisticated AI Chatbot</h1>
                        <p class="subtitle">Advanced Portfolio Demonstration</p>
                        <div>
                            <span class="feature-badge">RAG System</span>
                            <span class="feature-badge">Memory & Context</span>
                            <span class="feature-badge">Sentiment Analysis</span>
                            <span class="feature-badge">Dynamic Personalities</span>
                            <span class="feature-badge">Emotion Tracking</span>
                        </div>
                    </div>
                    """)

            # Main Chat Interface
            with gr.Row():
                with gr.Column(scale=2):
                    gr.HTML('<div class="panel-title">Intelligent Conversation</div>')

                    # Chat interface with custom styling
                    chatbot_interface = gr.Chatbot(
                        value=[],
                        height=600,
                        show_label=False,
                        elem_classes=["chatbot"],
                        bubble_full_width=False
                    )

                    # Message input with enhanced styling
                    with gr.Row():
                        message_input = gr.Textbox(
                            placeholder="Type your message here... Ask about AI, Python, Data Science, or anything else!",
                            lines=2,
                            max_lines=4,
                            scale=4,
                            show_label=False,
                            elem_classes=["gr-textbox"]
                        )
                        send_button = gr.Button(
                            "Send",
                            variant="primary",
                            scale=1,
                            elem_classes=["primary-button"]
                        )

                    # Quick action buttons
                    with gr.Row():
                        gr.Button("Tell me about AI", size="sm", variant="secondary")
                        gr.Button("Python tips", size="sm", variant="secondary")
                        gr.Button("Data Science", size="sm", variant="secondary")
                        clear_button = gr.Button("Clear Chat", size="sm", variant="secondary")

                # Control Panel
                with gr.Column(scale=1):
                    gr.HTML('<div class="panel-title">Control Panel</div>')

                    # Personality Selection
                    with gr.Group():
                        gr.HTML('<div class="metric-label">Personality Mode</div>')
                        personality_dropdown = gr.Dropdown(
                            choices=list(DEFAULT_PERSONALITIES.keys()),
                            value="friendly",
                            label="Select Personality",
                            elem_classes=["gr-dropdown"]
                        )
                        auto_personality = gr.Checkbox(
                            label="Auto-adapt based on emotion",
                            value=True
                        )

                    # User Settings
                    with gr.Group():
                        gr.HTML('<div class="metric-label">User Settings</div>')
                        user_name = gr.Textbox(
                            label="Your Name",
                            value="Demo User",
                            elem_classes=["gr-textbox"]
                        )
                        conversation_style = gr.Dropdown(
                            choices=["casual", "formal", "technical", "friendly"],
                            value="casual",
                            label="Conversation Style"
                        )

                    # System Status
                    with gr.Group():
                        gr.HTML('<div class="metric-label">System Status</div>')
                        system_status = gr.HTML('<div class="stat-item">Initializing...</div>')
                        refresh_status = gr.Button("Refresh Status", size="sm")

            # Analytics Dashboard
            with gr.Row():
                with gr.Column():
                    gr.HTML('<div class="panel-title">Analytics Dashboard</div>')

                    with gr.Row():
                        # Sentiment Analysis
                        with gr.Column():
                            gr.HTML('<div class="metric-label">Current Sentiment</div>')
                            sentiment_display = gr.HTML('<div class="stat-item">Neutral</div>')

                        # Emotion Tracking
                        with gr.Column():
                            gr.HTML('<div class="metric-label">Detected Emotion</div>')
                            emotion_display = gr.HTML('<div class="stat-item">Analyzing...</div>')

                        # Personality Used
                        with gr.Column():
                            gr.HTML('<div class="metric-label">Active Personality</div>')
                            personality_display = gr.HTML('<div class="stat-item">Friendly</div>')

                    # Conversation Insights
                    with gr.Row():
                        with gr.Column():
                            sentiment_chart = gr.Plot(label="Sentiment Timeline")
                        with gr.Column():
                            emotion_chart = gr.Plot(label="Emotion Distribution")

            # Knowledge Base Management
            with gr.Row():
                with gr.Column():
                    gr.HTML('<div class="panel-title">Knowledge Base Management</div>')

                    with gr.Row():
                        with gr.Column():
                            doc_input = gr.Textbox(
                                label="Add Knowledge",
                                placeholder="Paste text content to add to the knowledge base...",
                                lines=4,
                                elem_classes=["gr-textbox"]
                            )
                            doc_title = gr.Textbox(
                                label="Document Title",
                                placeholder="Enter a title for this knowledge...",
                                elem_classes=["gr-textbox"]
                            )
                            add_knowledge_button = gr.Button(
                                "Add to Knowledge Base",
                                variant="primary",
                                elem_classes=["primary-button"]
                            )

                        with gr.Column():
                            kb_stats = gr.HTML('<div class="stats-container">Loading knowledge base stats...</div>')
                            refresh_kb = gr.Button("Refresh KB Stats", size="sm")

            # Event Handlers
            def chat_response(message, history, personality, auto_adapt, user_name_val, style):
                if not message.strip():
                    return history, ""

                try:
                    # Update user profile
                    self.current_user_id = user_name_val.lower().replace(" ", "_")

                    # Get chatbot response
                    selected_personality = None if auto_adapt else personality
                    response_data = self.chatbot.chat(message, self.current_user_id, selected_personality)

                    # Update history
                    history.append([message, response_data['response']])

                    return history, ""

                except Exception as e:
                    logger.error(f"Chat interface error: {e}")
                    history.append([message, f"Error: {str(e)}"])
                    return history, ""

            def get_system_status():
                try:
                    stats = self.chatbot.get_system_stats()
                    memory_info = stats.get('memory_usage', {})

                    status_html = f"""
                    <div class="stats-container">
                        <div class="stat-item">
                            <div class="metric-value">{memory_info.get('allocated_gb', 0):.1f} GB</div>
                            <div class="metric-label">GPU Memory Used</div>
                        </div>
                        <div class="stat-item">
                            <div class="metric-value">{stats.get('active_conversations', 0)}</div>
                            <div class="metric-label">Active Conversations</div>
                        </div>
                        <div class="stat-item">
                            <div class="metric-value">Available</div>
                            <div class="metric-label">System Status</div>
                        </div>
                    </div>
                    """
                    return status_html
                except Exception as e:
                    return f'<div class="stat-item">Error: {str(e)}</div>'

            def get_kb_statistics():
                try:
                    rag_stats = self.chatbot.rag_system.get_rag_stats()
                    kb_info = rag_stats.get('knowledge_base', {})

                    stats_html = f"""
                    <div class="stats-container">
                        <div class="stat-item">
                            <div class="metric-value">{kb_info.get('total_chunks', 0)}</div>
                            <div class="metric-label">Knowledge Chunks</div>
                        </div>
                        <div class="stat-item">
                            <div class="metric-value">{kb_info.get('total_documents', 0)}</div>
                            <div class="metric-label">Documents</div>
                        </div>
                        <div class="stat-item">
                            <div class="metric-value">Ready</div>
                            <div class="metric-label">KB Status</div>
                        </div>
                    </div>
                    """
                    return stats_html
                except Exception as e:
                    return f'<div class="stat-item">Error loading KB stats: {str(e)}</div>'

            def add_knowledge_content(content, title):
                if not content.strip() or not title.strip():
                    return "Please provide both content and title."

                try:
                    self.chatbot.add_knowledge(content, title, {"type": "user_added"})
                    return f"Successfully added '{title}' to knowledge base!"
                except Exception as e:
                    return f"Error adding knowledge: {str(e)}"

            def create_sentiment_timeline():
                try:
                    emotion_summary = self.chatbot.get_user_emotion_summary(self.current_user_id)
                    if emotion_summary.get('status') == 'no_data':
                        fig = go.Figure()
                        fig.add_annotation(text="Start chatting to see sentiment analysis!",
                                         xref="paper", yref="paper", x=0.5, y=0.5)
                        fig.update_layout(title="Sentiment Timeline", template="plotly_dark")
                        return fig

                    # Create sample sentiment data for demo
                    import random
                    dates = pd.date_range(start='2024-01-01', periods=10, freq='D')
                    sentiments = [random.uniform(-0.5, 0.8) for _ in range(10)]

                    fig = go.Figure()
                    fig.add_trace(go.Scatter(x=dates, y=sentiments, mode='lines+markers',
                                           name='Sentiment Score', line=dict(color='#4ecdc4', width=3)))
                    fig.add_hline(y=0, line_dash="dash", line_color="white", annotation_text="Neutral")
                    fig.update_layout(title="Sentiment Over Time", template="plotly_dark",
                                    yaxis=dict(range=[-1, 1]), xaxis_title="Date", yaxis_title="Sentiment Score")
                    return fig
                except Exception as e:
                    fig = go.Figure()
                    fig.add_annotation(text=f"Error: {str(e)}", xref="paper", yref="paper", x=0.5, y=0.5)
                    return fig

            def create_emotion_pie():
                try:
                    # Sample emotion distribution for demo
                    emotions = ['joy', 'neutral', 'curiosity', 'satisfaction']
                    counts = [30, 25, 25, 20]
                    colors = ['#ff6b6b', '#4ecdc4', '#45b7d1', '#96ceb4']

                    fig = go.Figure(data=[go.Pie(labels=emotions, values=counts, hole=0.3,
                                               marker_colors=colors)])
                    fig.update_layout(title="Emotion Distribution", template="plotly_dark")
                    return fig
                except Exception as e:
                    fig = go.Figure()
                    fig.add_annotation(text=f"Error: {str(e)}", xref="paper", yref="paper", x=0.5, y=0.5)
                    return fig

            # Connect event handlers
            send_button.click(
                chat_response,
                inputs=[message_input, chatbot_interface, personality_dropdown, auto_personality, user_name, conversation_style],
                outputs=[chatbot_interface, message_input]
            )

            message_input.submit(
                chat_response,
                inputs=[message_input, chatbot_interface, personality_dropdown, auto_personality, user_name, conversation_style],
                outputs=[chatbot_interface, message_input]
            )

            clear_button.click(lambda: [], outputs=[chatbot_interface])

            refresh_status.click(get_system_status, outputs=[system_status])
            refresh_kb.click(get_kb_statistics, outputs=[kb_stats])

            add_knowledge_button.click(
                add_knowledge_content,
                inputs=[doc_input, doc_title],
                outputs=[gr.Textbox(visible=False)]
            )

            # Load initial data
            demo.load(get_system_status, outputs=[system_status])
            demo.load(get_kb_statistics, outputs=[kb_stats])
            demo.load(create_sentiment_timeline, outputs=[sentiment_chart])
            demo.load(create_emotion_pie, outputs=[emotion_chart])

        return demo

    def launch(self, share: bool = True, debug: bool = False):
        """Launch the Gradio interface"""
        interface = self.create_interface()
        interface.launch(
            share=share,
            debug=debug,
            server_name="0.0.0.0",
            server_port=7860,
            show_error=True,
            quiet=False
        )

# ***Testing Suite***

In [26]:
class TestRunner:
    """Comprehensive testing framework for the sophisticated chatbot system"""

    def __init__(self, chatbot: SophisticatedChatbot):
        self.chatbot = chatbot
        self.test_results = {}
        self.passed_tests = 0
        self.failed_tests = 0
        self.test_user_id = "test_user_12345"

    def log_test_result(self, test_name: str, passed: bool, message: str = "", execution_time: float = 0):
        """Log test results with detailed information"""
        status = "PASS" if passed else "FAIL"
        self.test_results[test_name] = {
            'status': status,
            'message': message,
            'execution_time': execution_time,
            'timestamp': datetime.now().isoformat()
        }

        if passed:
            self.passed_tests += 1
            logger.info(f"✓ {test_name}: {status} ({execution_time:.3f}s)")
        else:
            self.failed_tests += 1
            logger.error(f"✗ {test_name}: {status} - {message} ({execution_time:.3f}s)")

    def assert_test(self, condition: bool, test_name: str, success_msg: str, failure_msg: str, execution_time: float = 0):
        """Assert test condition and log result"""
        if condition:
            self.log_test_result(test_name, True, success_msg, execution_time)
        else:
            self.log_test_result(test_name, False, failure_msg, execution_time)
        return condition

In [27]:
class ComponentTester:
    """Tests individual components of the chatbot system"""

    def __init__(self, chatbot: SophisticatedChatbot, test_runner: TestRunner):
        self.chatbot = chatbot
        self.runner = test_runner

    def test_memory_system(self):
        """Test memory system functionality"""
        logger.info("Testing Memory System...")
        start_time = time.time()

        try:
            # Test user profile creation
            profile = self.chatbot.memory_system.create_user_profile("test_memory_user", "Test User")
            self.runner.assert_test(
                profile is not None and profile.user_id == "test_memory_user",
                "Memory: User Profile Creation",
                "User profile created successfully",
                "Failed to create user profile"
            )

            # Test message storage
            test_message = Message(
                id="test_msg_001",
                user_id="test_memory_user",
                content="This is a test message for memory system",
                timestamp=datetime.now(),
                message_type="user",
                sentiment_score=0.5,
                emotion="neutral",
                personality_used="friendly",
                context_sources=[],
                metadata={}
            )

            self.chatbot.memory_system.store_message(test_message)

            # Test conversation history retrieval
            history = self.chatbot.memory_system.db_manager.get_conversation_history("test_memory_user", limit=5)
            self.runner.assert_test(
                len(history) > 0 and history[0].content == test_message.content,
                "Memory: Message Storage and Retrieval",
                "Message stored and retrieved successfully",
                "Failed to store or retrieve message"
            )

            # Test semantic search
            if self.chatbot.memory_system.embedding_model:
                relevant_memories = self.chatbot.memory_system.retrieve_relevant_memories(
                    "test message", "test_memory_user", limit=3
                )
                self.runner.assert_test(
                    isinstance(relevant_memories, list),
                    "Memory: Semantic Search",
                    "Semantic search executed successfully",
                    "Semantic search failed"
                )

        except Exception as e:
            self.runner.log_test_result("Memory: System Test", False, f"Exception: {str(e)}")

        execution_time = time.time() - start_time
        logger.info(f"Memory system tests completed in {execution_time:.3f}s")

    def test_rag_system(self):
        """Test RAG system functionality"""
        logger.info("Testing RAG System...")
        start_time = time.time()

        try:
            # Test knowledge base initialization
            self.runner.assert_test(
                self.chatbot.rag_system.knowledge_base.is_loaded,
                "RAG: Knowledge Base Initialization",
                "Knowledge base initialized successfully",
                "Knowledge base failed to initialize"
            )

            # Test document processing
            test_content = "Artificial Intelligence is a transformative technology that enables machines to perform tasks requiring human intelligence."
            chunks = self.chatbot.rag_system.knowledge_base.document_processor.chunk_text(test_content)

            self.runner.assert_test(
                len(chunks) > 0 and isinstance(chunks[0], str),
                "RAG: Document Chunking",
                f"Document chunked into {len(chunks)} parts",
                "Document chunking failed"
            )

            # Test knowledge search
            search_results = self.chatbot.rag_system.knowledge_base.search_knowledge("artificial intelligence", top_k=3)

            self.runner.assert_test(
                isinstance(search_results, list),
                "RAG: Knowledge Search",
                f"Search returned {len(search_results)} results",
                "Knowledge search failed"
            )

            # Test context retrieval
            context = self.chatbot.rag_system.retrieve_and_format_context("What is machine learning?")

            self.runner.assert_test(
                isinstance(context, str),
                "RAG: Context Retrieval",
                "Context retrieved and formatted successfully",
                "Context retrieval failed"
            )

        except Exception as e:
            self.runner.log_test_result("RAG: System Test", False, f"Exception: {str(e)}")

        execution_time = time.time() - start_time
        logger.info(f"RAG system tests completed in {execution_time:.3f}s")

    def test_sentiment_analysis(self):
        """Test sentiment analysis functionality"""
        logger.info("Testing Sentiment Analysis...")
        start_time = time.time()

        try:
            # Test positive sentiment
            positive_text = "I am extremely happy and excited about this amazing project!"
            positive_result = self.chatbot.sentiment_analyzer.analyze_sentiment(positive_text)

            self.runner.assert_test(
                positive_result['sentiment_score'] > 0 and positive_result['sentiment_label'] == 'positive',
                "Sentiment: Positive Detection",
                f"Positive sentiment detected (score: {positive_result['sentiment_score']:.3f})",
                f"Failed to detect positive sentiment (score: {positive_result['sentiment_score']:.3f})"
            )

            # Test negative sentiment
            negative_text = "I am very disappointed and frustrated with this terrible situation."
            negative_result = self.chatbot.sentiment_analyzer.analyze_sentiment(negative_text)

            self.runner.assert_test(
                negative_result['sentiment_score'] < 0 and negative_result['sentiment_label'] == 'negative',
                "Sentiment: Negative Detection",
                f"Negative sentiment detected (score: {negative_result['sentiment_score']:.3f})",
                f"Failed to detect negative sentiment (score: {negative_result['sentiment_score']:.3f})"
            )

            # Test neutral sentiment
            neutral_text = "The weather today is okay and normal."
            neutral_result = self.chatbot.sentiment_analyzer.analyze_sentiment(neutral_text)

            self.runner.assert_test(
                abs(neutral_result['sentiment_score']) < 0.3,
                "Sentiment: Neutral Detection",
                f"Neutral sentiment detected (score: {neutral_result['sentiment_score']:.3f})",
                f"Failed to detect neutral sentiment (score: {neutral_result['sentiment_score']:.3f})"
            )

            # Test emotion detection
            emotion_tests = [
                ("I am so excited and happy!", "joy"),
                ("I am confused and don't understand.", "confusion"),
                ("I am angry and frustrated!", "anger"),
                ("I am scared and worried.", "fear")
            ]

            emotion_success = 0
            for text, expected_emotion in emotion_tests:
                result = self.chatbot.sentiment_analyzer.analyze_sentiment(text)
                if result['emotion'] == expected_emotion or result['emotion'] in ['joy', 'sadness', 'anger', 'fear', 'surprise', 'neutral']:
                    emotion_success += 1

            self.runner.assert_test(
                emotion_success >= len(emotion_tests) * 0.5,  # At least 50% accuracy
                "Sentiment: Emotion Detection",
                f"Emotion detection working ({emotion_success}/{len(emotion_tests)} correct)",
                f"Emotion detection failed ({emotion_success}/{len(emotion_tests)} correct)"
            )

        except Exception as e:
            self.runner.log_test_result("Sentiment: Analysis Test", False, f"Exception: {str(e)}")

        execution_time = time.time() - start_time
        logger.info(f"Sentiment analysis tests completed in {execution_time:.3f}s")

    def test_personality_system(self):
        """Test personality system functionality"""
        logger.info("Testing Personality System...")
        start_time = time.time()

        try:
            # Test personality retrieval
            for personality_name in DEFAULT_PERSONALITIES.keys():
                personality = self.chatbot.personality_manager.get_personality(personality_name)
                self.runner.assert_test(
                    personality is not None and personality.name is not None,
                    f"Personality: {personality_name.title()} Retrieval",
                    f"{personality_name.title()} personality loaded successfully",
                    f"Failed to load {personality_name} personality"
                )

            # Test personality recommendation
            test_context = {
                'sentiment_analysis': {
                    'emotion': 'sadness',
                    'sentiment_score': -0.7
                },
                'user_profile': UserProfile(
                    user_id="test_user",
                    name="Test User",
                    preferred_personality="analytical",
                    conversation_style="casual",
                    topics_of_interest=[],
                    sentiment_history=[],
                    personality_preferences={},
                    learning_goals=[],
                    interaction_patterns={},
                    created_at=datetime.now(),
                    last_active=datetime.now()
                )
            }

            recommended = self.chatbot.personality_manager.recommend_personality(test_context)
            self.runner.assert_test(
                recommended in DEFAULT_PERSONALITIES.keys(),
                "Personality: Recommendation System",
                f"Recommended {recommended} personality for sad user",
                "Personality recommendation failed"
            )

            # Test personality adaptation
            base_personality = self.chatbot.personality_manager.get_personality("friendly")
            adapted = self.chatbot.personality_manager.adapt_personality_traits(base_personality, test_context)

            self.runner.assert_test(
                adapted is not None and hasattr(adapted, 'traits'),
                "Personality: Trait Adaptation",
                "Personality traits adapted successfully",
                "Personality adaptation failed"
            )

        except Exception as e:
            self.runner.log_test_result("Personality: System Test", False, f"Exception: {str(e)}")

        execution_time = time.time() - start_time
        logger.info(f"Personality system tests completed in {execution_time:.3f}s")

    def test_model_manager(self):
        """Test model management functionality"""
        logger.info("Testing Model Manager...")
        start_time = time.time()

        try:
            # Test memory usage reporting
            memory_info = self.chatbot.model_manager.get_memory_usage()
            self.runner.assert_test(
                isinstance(memory_info, dict) and 'allocated_gb' in memory_info,
                "Model: Memory Usage Reporting",
                f"Memory usage: {memory_info.get('allocated_gb', 0):.1f} GB",
                "Memory usage reporting failed"
            )

            # Test model loading status
            model_loaded = self.chatbot.model_manager.current_primary_model is not None
            tokenizer_loaded = self.chatbot.model_manager.tokenizer is not None

            self.runner.assert_test(
                model_loaded and tokenizer_loaded,
                "Model: Loading Status",
                "Primary model and tokenizer loaded successfully",
                "Model or tokenizer loading failed"
            )

            # Test response generation (simple test)
            if model_loaded and tokenizer_loaded:
                test_personality = self.chatbot.personality_manager.get_personality("friendly")
                response = self.chatbot.model_manager.generate_response("Hello, how are you?", test_personality)

                self.runner.assert_test(
                    isinstance(response, str) and len(response) > 0,
                    "Model: Response Generation",
                    f"Generated response: {len(response)} characters",
                    "Response generation failed"
                )

        except Exception as e:
            self.runner.log_test_result("Model: Manager Test", False, f"Exception: {str(e)}")

        execution_time = time.time() - start_time
        logger.info(f"Model manager tests completed in {execution_time:.3f}s")

In [28]:
class IntegrationTester:
    """Tests end-to-end system integration"""

    def __init__(self, chatbot: SophisticatedChatbot, test_runner: TestRunner):
        self.chatbot = chatbot
        self.runner = test_runner

    def test_full_conversation_flow(self):
        """Test complete conversation flow from input to response"""
        logger.info("Testing Full Conversation Flow...")
        start_time = time.time()

        try:
            test_messages = [
                "Hello, can you tell me about artificial intelligence?",
                "I'm feeling excited about learning Python programming!",
                "I'm confused about machine learning algorithms.",
                "What are the best practices for data science?"
            ]

            successful_conversations = 0

            for i, message in enumerate(test_messages):
                try:
                    response_data = self.chatbot.chat(message, f"integration_test_user_{i}")

                    # Check response structure
                    required_fields = ['response', 'personality_used', 'user_sentiment', 'response_sentiment']
                    has_required_fields = all(field in response_data for field in required_fields)

                    # Check response quality
                    response_text = response_data.get('response', '')
                    has_valid_response = isinstance(response_text, str) and len(response_text) > 10

                    if has_required_fields and has_valid_response:
                        successful_conversations += 1
                        logger.info(f"Conversation {i+1}: SUCCESS - {len(response_text)} chars")
                    else:
                        logger.warning(f"Conversation {i+1}: PARTIAL - Missing fields or short response")

                except Exception as e:
                    logger.error(f"Conversation {i+1}: FAILED - {str(e)}")

            success_rate = successful_conversations / len(test_messages)
            self.runner.assert_test(
                success_rate >= 0.75,  # At least 75% success rate
                "Integration: Full Conversation Flow",
                f"Conversation flow working ({successful_conversations}/{len(test_messages)} successful)",
                f"Conversation flow issues ({successful_conversations}/{len(test_messages)} successful)"
            )

        except Exception as e:
            self.runner.log_test_result("Integration: Conversation Flow", False, f"Exception: {str(e)}")

        execution_time = time.time() - start_time
        logger.info(f"Integration tests completed in {execution_time:.3f}s")

    def test_system_initialization(self):
        """Test system initialization and component connectivity"""
        logger.info("Testing System Initialization...")
        start_time = time.time()

        try:
            # Test initialization status
            self.runner.assert_test(
                self.chatbot.initialization_complete,
                "Integration: System Initialization",
                "System initialized successfully",
                "System initialization incomplete"
            )

            # Test component connectivity
            components_connected = (
                self.chatbot.memory_system is not None and
                self.chatbot.rag_system is not None and
                self.chatbot.sentiment_analyzer is not None and
                self.chatbot.personality_manager is not None and
                self.chatbot.model_manager is not None
            )

            self.runner.assert_test(
                components_connected,
                "Integration: Component Connectivity",
                "All components connected successfully",
                "Component connectivity issues detected"
            )

            # Test RAG-Memory integration
            rag_memory_connected = self.chatbot.rag_system.memory_system is not None

            self.runner.assert_test(
                rag_memory_connected,
                "Integration: RAG-Memory Connection",
                "RAG and Memory systems connected",
                "RAG-Memory integration failed"
            )

        except Exception as e:
            self.runner.log_test_result("Integration: System Initialization", False, f"Exception: {str(e)}")

        execution_time = time.time() - start_time
        logger.info(f"System initialization tests completed in {execution_time:.3f}s")

In [29]:
class PerformanceTester:
    """Tests system performance and efficiency"""

    def __init__(self, chatbot: SophisticatedChatbot, test_runner: TestRunner):
        self.chatbot = chatbot
        self.runner = test_runner

    def test_response_time(self):
        """Test response time performance"""
        logger.info("Testing Response Time Performance...")

        test_messages = [
            "Hello!",
            "What is machine learning?",
            "Can you explain neural networks in detail?",
            "I need help with Python programming and data analysis."
        ]

        response_times = []

        for message in test_messages:
            start_time = time.time()
            try:
                self.chatbot.chat(message, "performance_test_user")
                response_time = time.time() - start_time
                response_times.append(response_time)
                logger.info(f"Response time for '{message[:30]}...': {response_time:.3f}s")
            except Exception as e:
                logger.warning(f"Performance test failed for message: {str(e)}")

        if response_times:
            avg_response_time = np.mean(response_times)
            max_response_time = max(response_times)

            self.runner.assert_test(
                avg_response_time < 30.0,  # Average under 30 seconds
                "Performance: Average Response Time",
                f"Average response time: {avg_response_time:.3f}s",
                f"Average response time too slow: {avg_response_time:.3f}s"
            )

            self.runner.assert_test(
                max_response_time < 60.0,  # Maximum under 60 seconds
                "Performance: Maximum Response Time",
                f"Maximum response time: {max_response_time:.3f}s",
                f"Maximum response time too slow: {max_response_time:.3f}s"
            )
        else:
            self.runner.log_test_result("Performance: Response Time", False, "No successful responses to measure")

    def test_memory_efficiency(self):
        """Test memory usage efficiency"""
        logger.info("Testing Memory Efficiency...")
        start_time = time.time()

        try:
            initial_memory = self.chatbot.model_manager.get_memory_usage()

            # Simulate multiple conversations
            for i in range(5):
                self.chatbot.chat(f"Test message {i} for memory efficiency testing.", f"memory_test_user_{i}")

            final_memory = self.chatbot.model_manager.get_memory_usage()

            memory_increase = final_memory.get('allocated_gb', 0) - initial_memory.get('allocated_gb', 0)

            self.runner.assert_test(
                memory_increase < 5.0,  # Less than 5GB increase
                "Performance: Memory Efficiency",
                f"Memory increase: {memory_increase:.3f}GB",
                f"Excessive memory increase: {memory_increase:.3f}GB"
            )

            # Test memory within limits
            current_memory = final_memory.get('allocated_gb', 0)
            total_memory = final_memory.get('total_gb', 40)
            memory_usage_percent = (current_memory / total_memory) * 100

            self.runner.assert_test(
                memory_usage_percent < 90,  # Under 90% memory usage
                "Performance: Memory Usage Limit",
                f"Memory usage: {memory_usage_percent:.1f}%",
                f"Memory usage too high: {memory_usage_percent:.1f}%"
            )

        except Exception as e:
            self.runner.log_test_result("Performance: Memory Efficiency", False, f"Exception: {str(e)}")

        execution_time = time.time() - start_time
        logger.info(f"Memory efficiency tests completed in {execution_time:.3f}s")

In [30]:
class ComprehensiveTestSuite:
    """Main test suite orchestrator"""

    def __init__(self, chatbot: SophisticatedChatbot):
        self.chatbot = chatbot
        self.test_runner = TestRunner(chatbot)

        # Initialize test classes
        self.component_tester = ComponentTester(chatbot, self.test_runner)
        self.integration_tester = IntegrationTester(chatbot, self.test_runner)
        self.performance_tester = PerformanceTester(chatbot, self.test_runner)

    def run_all_tests(self, include_performance: bool = True):
        """Run complete test suite"""
        logger.info("="*80)
        logger.info("STARTING COMPREHENSIVE CHATBOT TEST SUITE")
        logger.info("="*80)

        suite_start_time = time.time()

        # Component Tests
        logger.info("\n" + "="*50 + " COMPONENT TESTS " + "="*50)
        self.component_tester.test_memory_system()
        self.component_tester.test_rag_system()
        self.component_tester.test_sentiment_analysis()
        self.component_tester.test_personality_system()
        self.component_tester.test_model_manager()

        # Integration Tests
        logger.info("\n" + "="*50 + " INTEGRATION TESTS " + "="*50)
        self.integration_tester.test_system_initialization()
        self.integration_tester.test_full_conversation_flow()

        # Performance Tests (optional)
        if include_performance:
            logger.info("\n" + "="*50 + " PERFORMANCE TESTS " + "="*50)
            self.performance_tester.test_response_time()
            self.performance_tester.test_memory_efficiency()

        # Generate final report
        total_time = time.time() - suite_start_time
        self.generate_test_report(total_time)

    def generate_test_report(self, total_execution_time: float):
        """Generate comprehensive test report"""
        logger.info("\n" + "="*80)
        logger.info("TEST SUITE RESULTS")
        logger.info("="*80)

        total_tests = self.test_runner.passed_tests + self.test_runner.failed_tests
        success_rate = (self.test_runner.passed_tests / total_tests * 100) if total_tests > 0 else 0

        logger.info(f"Total Tests Run: {total_tests}")
        logger.info(f"Passed: {self.test_runner.passed_tests}")
        logger.info(f"Failed: {self.test_runner.failed_tests}")
        logger.info(f"Success Rate: {success_rate:.1f}%")
        logger.info(f"Total Execution Time: {total_execution_time:.3f}s")

        # Detailed results
        logger.info("\nDETAILED RESULTS:")
        logger.info("-" * 80)

        for test_name, result in self.test_runner.test_results.items():
            status_symbol = "✓" if result['status'] == 'PASS' else "✗"
            logger.info(f"{status_symbol} {test_name}: {result['status']} ({result['execution_time']:.3f}s)")
            if result['message']:
                logger.info(f"    {result['message']}")

        # Summary assessment
        logger.info("\n" + "="*80)
        if success_rate >= 90:
            logger.info("ASSESSMENT: EXCELLENT - System is production-ready")
        elif success_rate >= 75:
            logger.info("ASSESSMENT: GOOD - System is functional with minor issues")
        elif success_rate >= 50:
            logger.info("ASSESSMENT: FAIR - System needs significant improvements")
        else:
            logger.info("ASSESSMENT: POOR - System requires major fixes")

        logger.info("="*80)

        return {
            'total_tests': total_tests,
            'passed': self.test_runner.passed_tests,
            'failed': self.test_runner.failed_tests,
            'success_rate': success_rate,
            'execution_time': total_execution_time,
            'detailed_results': self.test_runner.test_results
        }

In [31]:
def run_chatbot_tests(chatbot: SophisticatedChatbot, include_performance: bool = True):
    """Convenience function to run all tests"""
    test_suite = ComprehensiveTestSuite(chatbot)
    return test_suite.run_all_tests(include_performance)

# ***System Initialization and Launch***

In [32]:
def initialize_chatbot_system(run_tests: bool = True, create_sample_kb: bool = True, launch_interface: bool = True):
    """
    Complete system initialization and launch

    Args:
        run_tests: Whether to run the comprehensive test suite
        create_sample_kb: Whether to create sample knowledge base
        launch_interface: Whether to launch the Gradio interface
    """

    logger.info("Starting Sophisticated Chatbot System...")
    logger.info("System specifications:")
    logger.info(f"- GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
    logger.info(f"- Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB" if torch.cuda.is_available() else "- Memory: N/A")
    logger.info(f"- PyTorch: {torch.__version__}")

    try:
        # Step 1: Initialize core chatbot system
        logger.info("Initializing core system...")

        config = ChatbotConfig()
        chatbot = SophisticatedChatbot(config)

        # Initialize all components
        chatbot.initialize(create_sample_kb=create_sample_kb)

        logger.info("Core system initialization completed")

        # Step 2: Run comprehensive tests (optional)
        if run_tests:
            logger.info("Running system tests...")

            test_results = run_chatbot_tests(chatbot, include_performance=True)

            if test_results:
                success_rate = test_results.get('success_rate', 0)
                if success_rate < 75:
                    logger.warning(f"Test success rate ({success_rate:.1f}%) is below recommended threshold")
                    response = input("Continue with system launch despite test issues? (y/n): ")
                    if response.lower() != 'y':
                        logger.info("System launch cancelled by user")
                        return None, None
                else:
                    logger.info(f"Tests passed with {success_rate:.1f}% success rate")

        # Step 3: Display system statistics
        system_stats = chatbot.get_system_stats()
        display_system_statistics(system_stats)

        # Step 4: Initialize Gradio interface
        interface = None
        if launch_interface:
            logger.info("Initializing Gradio interface...")

            interface = GradioInterface(chatbot)

            logger.info("System initialization complete")

        return chatbot, interface

    except Exception as e:
        logger.error(f"System initialization failed: {str(e)}")
        logger.error("Please check the error logs and try again")
        raise

In [33]:
def display_system_statistics(stats: Dict[str, Any]):
    """Display comprehensive system statistics"""

    memory_info = stats.get('memory_usage', {})
    rag_stats = stats.get('rag_stats', {})
    kb_info = rag_stats.get('knowledge_base', {})

    print("\nSYSTEM STATUS REPORT")
    print("-" * 40)

    # System Status
    print(f"System Status: {'READY' if stats.get('initialization_complete') else 'INITIALIZING'}")
    print(f"Active Conversations: {stats.get('active_conversations', 0)}")
    print(f"Available Personalities: {len(stats.get('personalities_available', []))}")

    # Memory Information
    if memory_info:
        print(f"\nGPU Memory Usage:")
        print(f"  Allocated: {memory_info.get('allocated_gb', 0):.2f} GB")
        print(f"  Total: {memory_info.get('total_gb', 0):.2f} GB")

        usage_percent = (memory_info.get('allocated_gb', 0) / memory_info.get('total_gb', 1)) * 100
        print(f"  Usage: {usage_percent:.1f}%")

    # Knowledge Base Information
    if kb_info:
        print(f"\nKnowledge Base:")
        print(f"  Documents: {kb_info.get('total_documents', 0)}")
        print(f"  Chunks: {kb_info.get('total_chunks', 0)}")
        print(f"  Status: {'LOADED' if kb_info.get('is_loaded') else 'NOT LOADED'}")

    print("-" * 40)

In [34]:
def launch_system_interface(chatbot: SophisticatedChatbot, interface: GradioInterface):
    """Demonstrate key chatbot capabilities with sample interactions"""

    print("\n" + "="*80)
    print("DEMONSTRATING CHATBOT CAPABILITIES")
    print("="*80)

    demo_conversations = [
        {
            "title": "AI Knowledge Query",
            "message": "What is machine learning and how does it work?",
            "expected_features": ["RAG retrieval", "Knowledge synthesis", "Professional tone"]
        },
        {
            "title": "Emotional Interaction",
            "message": "I'm feeling really frustrated with my Python code and nothing is working!",
            "expected_features": ["Sentiment analysis", "Emotion detection", "Empathetic response"]
        },
        {
            "title": "Technical Question",
            "message": "Can you explain the difference between supervised and unsupervised learning?",
            "expected_features": ["Knowledge base search", "Analytical personality", "Detailed explanation"]
        }
    ]

    for i, demo in enumerate(demo_conversations, 1):
        print(f"\nDEMO {i}: {demo['title']}")
        print("-" * 60)
        print(f"User Message: {demo['message']}")
        print(f"Expected Features: {', '.join(demo['expected_features'])}")

        try:
            start_time = time.time()
            response_data = chatbot.chat(demo['message'], f"demo_user_{i}")
            response_time = time.time() - start_time

            print(f"\nBot Response ({response_time:.2f}s):")
            print(f"Personality Used: {response_data.get('personality_used', 'Unknown')}")
            print(f"User Sentiment: {response_data.get('user_sentiment', {}).get('sentiment_label', 'Unknown')} "
                  f"({response_data.get('user_sentiment', {}).get('sentiment_score', 0):.3f})")
            print(f"Detected Emotion: {response_data.get('user_sentiment', {}).get('emotion', 'Unknown')}")
            print(f"Context Used: {'Yes' if response_data.get('context_used') else 'No'}")
            print(f"Response: {response_data.get('response', 'No response generated')[:200]}...")

        except Exception as e:
            print(f"Demo failed: {str(e)}")

        print("-" * 60)

In [35]:
def quick_interactive_test(chatbot: SophisticatedChatbot):
    """Quick interactive test for manual validation"""

    print("\n" + "="*80)
    print("QUICK INTERACTIVE TEST")
    print("="*80)
    print("Type messages to test the chatbot. Type 'quit' to exit.")
    print("Try different emotions and topics to see personality adaptation!")

    user_id = "interactive_test_user"
    conversation_count = 0

    while True:
        try:
            user_input = input(f"\nYou: ").strip()

            if user_input.lower() in ['quit', 'exit', 'stop']:
                print("Interactive test ended.")
                break

            if not user_input:
                continue

            start_time = time.time()
            response_data = chatbot.chat(user_input, user_id)
            response_time = time.time() - start_time

            print(f"\nBot ({response_data.get('personality_used', 'unknown')} personality, {response_time:.2f}s):")
            print(f"{response_data.get('response', 'No response')}")

            # Show analysis details
            sentiment = response_data.get('user_sentiment', {})
            print(f"\n[Analysis: {sentiment.get('sentiment_label', 'unknown')} sentiment "
                  f"({sentiment.get('sentiment_score', 0):.2f}), {sentiment.get('emotion', 'unknown')} emotion]")

            conversation_count += 1

        except KeyboardInterrupt:
            print("\nInteractive test interrupted by user.")
            break
        except Exception as e:
            print(f"Error during interaction: {str(e)}")

    print(f"\nCompleted {conversation_count} test conversations.")

In [36]:
def demonstrate_chatbot_capabilities(chatbot: SophisticatedChatbot):
    """Demonstrate key chatbot capabilities with sample interactions"""

    logger.info("Running capability demonstrations...")

    demo_conversations = [
        {
            "title": "AI Knowledge Query",
            "message": "What is machine learning and how does it work?",
        },
        {
            "title": "Emotional Interaction",
            "message": "I'm feeling really frustrated with my Python code and nothing is working!",
        },
        {
            "title": "Technical Question",
            "message": "Can you explain the difference between supervised and unsupervised learning?",
        }
    ]

    for i, demo in enumerate(demo_conversations, 1):
        logger.info(f"Demo {i}: {demo['title']}")
        logger.info(f"User: {demo['message']}")

        try:
            start_time = time.time()
            response_data = chatbot.chat(demo['message'], f"demo_user_{i}")
            response_time = time.time() - start_time

            logger.info(f"Bot ({response_data.get('personality_used', 'unknown')}, {response_time:.2f}s):")
            logger.info(f"Response: {response_data.get('response', 'No response')[:150]}...")
            logger.info(f"Sentiment: {response_data.get('user_sentiment', {}).get('sentiment_label', 'unknown')} "
                       f"({response_data.get('user_sentiment', {}).get('emotion', 'unknown')})")

        except Exception as e:
            logger.error(f"Demo {i} failed: {str(e)}")

        logger.info("-" * 50)
    """Launch the Gradio interface with pre-launch checks"""

    logger.info("Launching Gradio interface...")

    # Pre-launch system check
    checks = [
        ("System initialization", chatbot.initialization_complete),
        ("Model manager ready", chatbot.model_manager.current_primary_model is not None),
        ("Memory system ready", chatbot.memory_system.embedding_model is not None),
        ("RAG system ready", chatbot.rag_system.knowledge_base.is_loaded),
        ("Sentiment analyzer ready", chatbot.sentiment_analyzer.sentiment_pipeline is not None)
    ]

    all_checks_passed = True
    for check_name, check_result in checks:
        status = "PASS" if check_result else "FAIL"
        logger.info(f"  {check_name}: {status}")
        if not check_result:
            all_checks_passed = False

    if not all_checks_passed:
        logger.warning("Some system checks failed. Interface may have limited functionality.")
        response = input("Continue with launch? (y/n): ")
        if response.lower() != 'y':
            logger.info("Interface launch cancelled.")
            return

    logger.info("Launching interface...")

    try:
        interface.launch(share=True, debug=False)
    except Exception as e:
        logger.error(f"Failed to launch interface: {str(e)}")
        logger.info("You can still use the chatbot programmatically.")

In [37]:
# Main execution function
def main():
    """Main function to orchestrate the entire system"""

    logger.info("Initializing Sophisticated Chatbot System")

    # Configuration options
    run_comprehensive_tests = True
    create_sample_knowledge = True
    launch_gradio_interface = True
    run_demonstrations = False  # Set to True to run capability demos

    try:
        # Initialize the complete system
        chatbot, interface = initialize_chatbot_system(
            run_tests=run_comprehensive_tests,
            create_sample_kb=create_sample_knowledge,
            launch_interface=launch_gradio_interface
        )

        if chatbot is None:
            logger.error("System initialization failed")
            return

        # Optional demonstrations for validation
        if run_demonstrations:
            demonstrate_chatbot_capabilities(chatbot)

        # Launch interface if requested
        if launch_gradio_interface and interface:
            launch_system_interface(chatbot, interface)

        logger.info("System initialization complete")

        return chatbot, interface

    except Exception as e:
        logger.error(f"System launch failed: {str(e)}")
        return None, None

In [38]:
# Execute the main function
if __name__ == "__main__":
    chatbot_system, gradio_interface = main()
else:
    # If imported as module, provide direct access
    logger.info("Sophisticated Chatbot module imported successfully")

2025-08-02 08:12:36,952 - SophisticatedChatbot - INFO - Initializing Sophisticated Chatbot System
2025-08-02 08:12:36,953 - SophisticatedChatbot - INFO - Starting Sophisticated Chatbot System...
2025-08-02 08:12:36,954 - SophisticatedChatbot - INFO - System specifications:
2025-08-02 08:12:36,954 - SophisticatedChatbot - INFO - - GPU: NVIDIA A100-SXM4-40GB
2025-08-02 08:12:36,955 - SophisticatedChatbot - INFO - - Memory: 42.5 GB
2025-08-02 08:12:36,956 - SophisticatedChatbot - INFO - - PyTorch: 2.6.0+cu124
2025-08-02 08:12:36,956 - SophisticatedChatbot - INFO - Initializing core system...
2025-08-02 08:12:57,793 - SophisticatedChatbot - INFO - Google Drive mounted successfully


Mounted at /content/drive


2025-08-02 08:12:59,320 - SophisticatedChatbot - INFO - Database initialized successfully
2025-08-02 08:13:00,528 - SophisticatedChatbot - INFO - Vector database initialized successfully
2025-08-02 08:13:00,536 - SophisticatedChatbot - INFO - Database initialized successfully
2025-08-02 08:13:00,537 - SophisticatedChatbot - INFO - Sophisticated Chatbot initialized
2025-08-02 08:13:00,537 - SophisticatedChatbot - INFO - Initializing chatbot components...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...


Bot Response (16.25s):
Personality Used: friendly
User Sentiment: neutral (0.000)
Detected Emotion: neutral
Context Used: Yes
Response: Machine Learning is a subset of Artificial Intelligence that allows systems to learn and improve from experience without being explicitly programmed. This means instead of being given specific rules f...
------------------------------------------------------------

DEMO 2: Emotional Interaction
------------------------------------------------------------
User Message: I'm feeling really frustrated with my Python code and nothing is working!
Expected Features: Sentiment analysis, Emotion detection, Empathetic response


2025-08-02 08:21:12,935 - SophisticatedChatbot - INFO - User profile saved: demo_user_2
2025-08-02 08:21:12,994 - SophisticatedChatbot - INFO - User profile saved: demo_user_2
2025-08-02 08:21:12,995 - SophisticatedChatbot - INFO - Chat response generated for user demo_user_2
2025-08-02 08:21:13,039 - SophisticatedChatbot - INFO - User profile saved: demo_user_3



Bot Response (14.09s):
Personality Used: friendly
User Sentiment: neutral (0.000)
Detected Emotion: anger
Context Used: Yes
Response: Don't worry, we can work through this together. Let me see if I can help you identify the issue. Can you tell me what specific problem you're encountering? For example, is there a particular error mes...
------------------------------------------------------------

DEMO 3: Technical Question
------------------------------------------------------------
User Message: Can you explain the difference between supervised and unsupervised learning?
Expected Features: Knowledge base search, Analytical personality, Detailed explanation


2025-08-02 08:21:20,422 - SophisticatedChatbot - INFO - User profile saved: demo_user_3
2025-08-02 08:21:20,499 - SophisticatedChatbot - INFO - User profile saved: demo_user_3
2025-08-02 08:21:20,500 - SophisticatedChatbot - INFO - Chat response generated for user demo_user_3
2025-08-02 08:21:20,501 - SophisticatedChatbot - INFO - System initialization complete



Bot Response (7.50s):
Personality Used: friendly
User Sentiment: neutral (0.000)
Detected Emotion: neutral
Context Used: Yes
Response: Of course! Supervised learning is when we have labeled training data - this means the correct output or answer for each example is already provided. The goal is for the machine learning algorithm to l...
------------------------------------------------------------


In [39]:
# Launch the interface manually
if 'chatbot_system' in globals() and chatbot_system is not None:
    interface = GradioInterface(chatbot_system)
    interface.launch(share=True, debug=False)
else:
    print("Chatbot system not initialized. Run Cell 9 first.")

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://52a6f6f54e773340e1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
